In [1]:
!pip install beautifulsoup4 requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def get_carousell_selenium():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 无头模式
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        driver.get("https://www.carousell.com.hk/search/dimoo?addRecent=true&canChangeKeyword=true&includeSuggestions=true&t-search_query_source=direct_search")
        time.sleep(3)  # 等待页面加载
        
        # 检查页面标题或特定元素确认加载成功
        if "Carousell" in driver.title:
            print("success")
            page_source = driver.page_source
            return page_source
        else:
            print("页面加载异常")
            return None
            
    except Exception as e:
        print(f"Selenium 错误: {e}")
        return None
    finally:
        driver.quit()

# 使用Selenium
# result = get_carousell_selenium()

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime

def scrape_carousell_labubu():
    chrome_options = Options()
    # 先去掉无头模式进行调试
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    all_products = []
    max_pages = 10  # 设置最大翻页次数
    
    try:
        # 直接访问搜索页面
        search_url = "https://www.carousell.com.hk/search/molly?addRecent=true&canChangeKeyword=true&includeSuggestions=true&t-search_query_source=direct_search"
        print(f"正在访问搜索页面: {search_url}")
        driver.get(search_url)
        
        # 使用显式等待
        wait = WebDriverWait(driver, 15)
        
        # 等待页面加载
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        print("页面body加载完成")
        
        current_page = 1
        
        while current_page <= max_pages:
            print(f"\n=== 正在处理第 {current_page} 页 ===")
            
            # 滚动页面以加载所有内容
            scroll_page(driver)
            
            # 提取当前页面商品信息
            products = extract_products(driver)
            if products:
                all_products.extend(products)
                print(f"第 {current_page} 页找到 {len(products)} 个商品")
            else:
                print(f"第 {current_page} 页未找到商品")
            
            # 尝试点击"顯示更多結果"按钮
            if not click_show_more_button(driver, wait):
                print("没有找到'顯示更多結果'按钮，可能已到最后一页")
                break
            
            current_page += 1
            time.sleep(3)  # 等待新内容加载
            
        print(f"\n总共爬取 {len(all_products)} 个商品")
        
        # 保存数据
        save_data(all_products)
        
        return all_products
        
    except Exception as e:
        print(f"爬取过程中出错: {e}")
        # 即使出错也尝试保存已收集的数据
        if all_products:
            save_data(all_products)
        return None
    finally:
        print("关闭浏览器...")
        driver.quit()

def scroll_page(driver):
    """滚动页面以加载所有内容"""
    print("开始滚动页面...")
    
    # 获取页面初始高度
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 多次滚动以确保加载所有内容
    for i in range(3):
        # 滚动到页面底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"滚动 {i+1}/3 次")
        time.sleep(2)
        
        # 计算新的滚动高度
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 如果高度没有变化，停止滚动
        if new_height == last_height:
            break
        
        last_height = new_height

def extract_products(driver):
    """提取商品信息"""
    products = []
    
    # 尝试多种可能的产品选择器
    product_selectors = [
        '[data-testid*="product"]',
        '[data-testid*="listing"]',
        '[class*="productCard"]',
        '[class*="listing"]',
        'div[class*="D_"]',  # Carousell 的动态类名
        'a[href*="/p/"]',    # 商品链接
        '[class*="D_pA"]',   # 商品卡片类
        '[class*="D_qh"]'    # 另一个可能的商品卡片类
    ]
    
    for selector in product_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            if elements:
                print(f"使用选择器 '{selector}' 找到 {len(elements)} 个元素")
                
                for i, element in enumerate(elements):
                    try:
                        product_info = extract_product_info(element, driver)
                        if product_info and product_info['title']:
                            # 检查是否重复
                            if not any(p['title'] == product_info['title'] and p['price'] == product_info['price'] for p in products):
                                products.append(product_info)
                    except Exception as e:
                        continue  # 跳过单个商品提取错误
                
                if products:
                    break
        except Exception as e:
            continue
    
    return products

def extract_product_info(element, driver):
    """提取单个商品信息 - 包括商品名称、价格、发布时间、发布用户"""
    try:
        # 获取商品文本内容
        text = element.text.strip()
        if not text or len(text) < 10:  # 过滤掉太短的内容
            return None
        
        # 获取商品链接
        link = ""
        try:
            if element.tag_name == "a":
                link = element.get_attribute("href")
            else:
                link_element = element.find_element(By.CSS_SELECTOR, "a")
                link = link_element.get_attribute("href")
        except:
            pass
        
        # 提取各项信息
        title = extract_title(text)
        price = extract_price(text)
        seller = extract_seller(text)
        post_time = extract_post_time(text)
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        print(f"提取商品信息时出错: {e}")
        return None

def extract_price(text):
    """从文本中提取价格"""
    import re
    # 匹配 HK$ 或 $ 开头的价格
    price_patterns = [
        r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)',
        r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(HKD|港幣)'
    ]
    
    for pattern in price_patterns:
        match = re.search(pattern, text)
        if match:
            if len(match.groups()) >= 2:
                return f"{match.group(1)}{match.group(2)}"
    return ""

def extract_title(text):
    """从文本中提取标题"""
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if (line and 
            not any(keyword in line for keyword in ['HK$', '$', '•', 'Like', '留言', '顯示更多結果', 'HKD', '港幣']) and
            len(line) > 3 and
            not re.search(r'\d{1,2}\s*(小時|天|分鐘|月|年)前', line) and  # 排除时间信息
            not re.search(r'@\w+', line)):  # 排除用户名
            return line[:150]  # 限制标题长度
    return text.split('\n')[0][:150] if text else ""

def extract_seller(text):
    """从文本中提取发布用户"""
    import re
    # 匹配@开头的用户名
    username_pattern = r'@(\w+)'
    match = re.search(username_pattern, text)
    if match:
        return match.group(1)
    
    # 如果没有@符号，尝试从文本中提取可能的用户名
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        # 排除明显不是用户名的行
        if (line and 
            not any(keyword in line for keyword in ['HK$', '$', 'HKD', '港幣', 'Like', '留言']) and
            not re.search(r'\d{1,2}\s*(小時|天|分鐘|月|年)前', line) and
            len(line) < 20):  # 用户名通常不会太长
            return line
    
    return ""

def extract_post_time(text):
    """从文本中提取发布时间"""
    import re
    # 匹配时间格式：X小時前, X天前, X分鐘前等
    time_patterns = [
        r'(\d{1,2})\s*小時前',
        r'(\d{1,2})\s*天前',
        r'(\d{1,2})\s*分鐘前',
        r'(\d{1,2})\s*月前',
        r'(\d{1,2})\s*年前',
        r'(\d{1,2})\s*hours?\s*ago',
        r'(\d{1,2})\s*days?\s*ago',
        r'(\d{1,2})\s*months?\s*ago',
        r'(\d{1,2})\s*years?\s*ago'
    ]
    
    for pattern in time_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(0)
    
    return ""

def click_show_more_button(driver, wait):
    """点击'顯示更多結果'按钮"""
    try:
        # 方法1: 通过按钮文本内容查找
        show_more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), '顯示更多結果')]")
        
        for button in show_more_buttons:
            if button.is_displayed() and button.is_enabled():
                # 滚动到按钮位置
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                time.sleep(1)
                
                # 点击按钮
                driver.execute_script("arguments[0].click();", button)
                print("成功点击'顯示更多結果'按钮")
                return True
        
        # 方法2: 通过CSS类名查找
        show_more_selectors = [
            'button[class*="D_mW"]',  # 包含D_mW类的按钮
            'button[type="button"]'   # 所有按钮类型的元素
        ]
        
        for selector in show_more_selectors:
            try:
                buttons = driver.find_elements(By.CSS_SELECTOR, selector)
                for button in buttons:
                    if button.is_displayed() and button.is_enabled() and "顯示更多結果" in button.text:
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", button)
                        print("通过CSS选择器点击'顯示更多結果'按钮")
                        return True
            except:
                continue
        
        print("未找到可点击的'顯示更多結果'按钮")
        return False
        
    except Exception as e:
        print(f"点击'顯示更多結果'按钮时出错: {e}")
        return False

def save_data(products):
    """保存数据到文件 - 修复文件权限问题"""
    if not products:
        print("没有数据可保存")
        return
    
    # 去重
    unique_products = []
    seen_items = set()
    for product in products:
        # 使用标题和价格作为唯一标识
        item_key = f"{product['title']}_{product['price']}"
        if item_key not in seen_items:
            unique_products.append(product)
            seen_items.add(item_key)
    
    print(f"去重后剩余 {len(unique_products)} 个商品")
    
    try:
        # 尝试在当前目录保存
        current_dir = os.getcwd()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # CSV文件
        csv_filename = f'carousell_labubu_products_{timestamp}.csv'
        df = pd.DataFrame(unique_products)
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"数据已保存到 {csv_filename}")
        
        # JSON文件
        json_filename = f'carousell_labubu_products_{timestamp}.json'
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(unique_products, f, ensure_ascii=False, indent=2)
        print(f"数据已保存到 {json_filename}")
        
    except Exception as e:
        print(f"文件保存失败: {e}")
        print("尝试在用户目录保存...")
        
        try:
            # 尝试在用户目录保存
            user_dir = os.path.expanduser("~")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # CSV文件
            csv_filename = os.path.join(user_dir, f'carousell_labubu_products_{timestamp}.csv')
            df = pd.DataFrame(unique_products)
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            print(f"数据已保存到 {csv_filename}")
            
            # JSON文件
            json_filename = os.path.join(user_dir, f'carousell_labubu_products_{timestamp}.json')
            with open(json_filename, 'w', encoding='utf-8') as f:
                json.dump(unique_products, f, ensure_ascii=False, indent=2)
            print(f"数据已保存到 {json_filename}")
            
        except Exception as e2:
            print(f"用户目录保存也失败: {e2}")
            print("最后尝试：只在控制台显示结果")
            display_results(unique_products)

def display_results(products):
    """在控制台显示结果"""
    print("\n" + "="*80)
    print("爬取结果（前20个商品）:")
    print("="*80)
    
    for i, product in enumerate(products[:20]):
        print(f"\n{i+1}. 商品名称: {product['title']}")
        print(f"   价格: {product['price']}")
        print(f"   发布用户: {product['seller']}")
        print(f"   发布时间: {product['post_time']}")
        print(f"   商品链接: {product['link']}")
    
    print(f"\n总共爬取 {len(products)} 个商品")
    print("由于文件权限问题，数据未保存到文件")

# 导入re模块（在文件顶部添加）
import re

# 测试执行
if __name__ == "__main__":
    products = scrape_carousell_labubu()
    if products:
        print(f"成功爬取 {len(products)} 个商品")
    else:
        print("爬取失败")

正在访问搜索页面: https://www.carousell.com.hk/search/molly?addRecent=true&canChangeKeyword=true&includeSuggestions=true&t-search_query_source=direct_search
页面body加载完成

=== 正在处理第 1 页 ===
开始滚动页面...
滚动 1/3 次
滚动 2/3 次
使用选择器 '[data-testid*="listing"]' 找到 144 个元素
使用选择器 '[class*="listing"]' 找到 1 个元素
第 1 页找到 1 个商品
成功点击'顯示更多結果'按钮

=== 正在处理第 2 页 ===
开始滚动页面...
滚动 1/3 次
使用选择器 '[data-testid*="listing"]' 找到 288 个元素
第 2 页找到 145 个商品
成功点击'顯示更多結果'按钮

=== 正在处理第 3 页 ===
开始滚动页面...
滚动 1/3 次
使用选择器 '[data-testid*="listing"]' 找到 426 个元素
第 3 页找到 199 个商品
成功点击'顯示更多結果'按钮

=== 正在处理第 4 页 ===
开始滚动页面...
滚动 1/3 次
使用选择器 '[data-testid*="listing"]' 找到 567 个元素
第 4 页找到 254 个商品
成功点击'顯示更多結果'按钮

=== 正在处理第 5 页 ===
开始滚动页面...
滚动 1/3 次
使用选择器 '[data-testid*="listing"]' 找到 711 个元素
第 5 页找到 320 个商品
成功点击'顯示更多結果'按钮

=== 正在处理第 6 页 ===
开始滚动页面...
滚动 1/3 次
使用选择器 '[data-testid*="listing"]' 找到 855 个元素
第 6 页找到 374 个商品
成功点击'顯示更多結果'按钮

=== 正在处理第 7 页 ===
开始滚动页面...
滚动 1/3 次
使用选择器 '[data-testid*="listing"]' 找到 999 个元素
第 7 页找到 437 个商品
成功点击'顯示更多結果'按钮

=== 正

In [19]:
#v3
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime
import re
import hashlib
import glob

def scrape_carousell_labubu():
    chrome_options = Options()
    # 先去掉无头模式进行调试
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    all_products = []
    max_pages = 10  # 减少页数进行测试
    seen_products = set()
    
    try:
        # 直接访问搜索页面
        search_url = "https://www.carousell.com.hk/search/labubu?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search"
        print(f"正在访问搜索页面: {search_url}")
        driver.get(search_url)
        
        # 使用显式等待
        wait = WebDriverWait(driver, 15)
        
        # 等待页面加载
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        print("页面body加载完成")
        
        current_page = 1
        
        while current_page <= max_pages:
            print(f"\n=== 正在处理第 {current_page} 页 ===")
            
            # 滚动页面以加载所有内容
            scroll_page(driver)
            
            # 等待新内容加载
            time.sleep(2)
            
            # 提取当前页面商品信息 - 使用新的方法
            products = extract_products_new_method(driver, seen_products)
            if products:
                all_products.extend(products)
                print(f"第 {current_page} 页找到 {len(products)} 个新商品")
                print(f"累计找到 {len(all_products)} 个唯一商品")
            else:
                print(f"第 {current_page} 页未找到新商品")
            
            # 检查是否还有更多内容
            if not click_show_more_button(driver, wait):
                print("没有找到'顯示更多結果'按钮，可能已到最后一页")
                break
            
            current_page += 1
            time.sleep(3)
            
        print(f"\n爬取完成！总共找到 {len(all_products)} 个唯一商品")
        
        # 保存数据到桌面
        saved_file = save_data_to_desktop(all_products)
        
        return all_products, saved_file
        
    except Exception as e:
        print(f"爬取过程中出错: {e}")
        # 即使出错也尝试保存已收集的数据
        if all_products:
            saved_file = save_data_to_desktop(all_products)
            return all_products, saved_file
        return None, None
    finally:
        print("关闭浏览器...")
        driver.quit()

# 找到整个 scroll_page 函数（大约在V3代码的第120行左右）
# 用这个改进版本替换整个函数：

def scroll_page(driver):
    """改进的页面滚动"""
    print("开始滚动页面...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 增加滚动次数和间隔
    for i in range(5):  # 增加到5次
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"滚动 {i+1}/5 次")
        time.sleep(2)  # 增加到2秒
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("页面高度未变化，停止滚动")
            break
        last_height = new_height
    
    # 额外向上滚动一点触发懒加载
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 500);")
    time.sleep(1)

def extract_products_new_method(driver, seen_products):
    """新的商品信息提取方法 - 直接从商品卡片结构提取"""
    products = []
    
    try:
        # 方法1: 通过商品卡片容器查找
        product_containers = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid*="listing-card"], div[class*="D_pA"], div[class*="D_qh"], div[class*="D_qW"], a[href*="/p/"]')
        
        print(f"🔍 找到 {len(product_containers)} 个商品容器")
        
        valid_count = 0
        for i, container in enumerate(product_containers):
            try:
                product_info = extract_product_from_container(container)
                if product_info and product_info['title']:
                    # 创建商品唯一标识
                    product_id = create_product_id(product_info)
                    
                    # 检查是否重复
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        valid_count += 1
                        print(f"✅ [{i+1}/{len(product_containers)}] 成功提取: {product_info['title'][:40]}... | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
                else:
                    print(f"❌ [{i+1}/{len(product_containers)}] 无效商品")
            except Exception as e:
                print(f"⚠️ [{i+1}/{len(product_containers)}] 处理失败: {e}")
                continue
        
        print(f"📊 有效商品: {valid_count}/{len(product_containers)}")
        
        # 如果方法1没找到，使用方法2
        if not products:
            products = extract_products_by_structure(driver, seen_products)
            
    except Exception as e:
        print(f"提取商品时出错: {e}")
    
    return products

def extract_product_from_container(container):
    """从商品容器提取完整信息"""
    try:
        # 获取容器内所有文本
        container_text = container.text.strip()
        if not container_text or len(container_text) < 20:
            return None
        
        # 提取商品名称 - 查找标题元素
        title = ""
        title_selectors = [
            'p.D_lX.D_lY.D_mc.D_mg.D_mj.D_ml.D_mh.D_lU',
            'p[class*="D_lX"]',
            'h3', 'h4', 'p'
        ]
        
        for selector in title_selectors:
            try:
                title_elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in title_elements:
                    text = element.text.strip()
                    if text and len(text) > 2:
                        title = text
                        break
                if title:
                    break
            except:
                continue
        
        if not title:
            # 从文本中提取可能的标题
            lines = container_text.split('\n')
            for line in lines:
                if 'labubu' in line.lower() and len(line) > 3:
                    title = line
                    break
        
        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"
        
        # 提取发布用户 - 直接在整个容器中搜索
        seller = ""
        # 方法1: 通过data-testid查找
        try:
            seller_elements = container.find_elements(By.CSS_SELECTOR, '[data-testid="listing-card-text-seller-name"]')
            for element in seller_elements:
                seller_text = element.text.strip()
                if seller_text:
                    seller = seller_text
                    break
        except:
            pass
        
        # 方法2: 查找特定类名的元素
        if not seller:
            try:
                seller_elements = container.find_elements(By.CSS_SELECTOR, 'p.D_lX.M_ml.D_lY.M_mm.D_mc.M_mq.D_mf.M_ms.D_mj.M_mw.D_mm.M_mz.D_lU')
                for element in seller_elements:
                    seller_text = element.text.strip()
                    if seller_text:
                        seller = seller_text
                        break
            except:
                pass
        
        # 方法3: 从文本中分析可能的用户名
        if not seller:
            lines = container_text.split('\n')
            for line in lines:
                line = line.strip()
                if (line and 
                    not re.search(r'(HK\$|\$|\d+)', line) and
                    not re.search(r'\d+\s*(小時|天|分鐘|月|年)前', line) and
                    line != title and
                    len(line) > 1 and len(line) < 25 and
                    not any(keyword in line.lower() for keyword in ['labubu', 'like', '留言', '立即聊聊'])):
                    seller = line
                    break
        
        # 提取发布时间
        post_time = ""
        # 方法1: 通过特定样式查找
        try:
            time_elements = container.find_elements(By.CSS_SELECTOR, 'p[style*="color:#00bfa2"]')
            for element in time_elements:
                time_text = element.text.strip()
                if time_text and re.search(r'\d+\s*(小時|天|分鐘|月|年)前', time_text):
                    post_time = time_text
                    break
        except:
            pass
        
        # 方法2: 查找特定类名的元素
        if not post_time:
            try:
                time_elements = container.find_elements(By.CSS_SELECTOR, 'p.D_lX.M_ml.D_lY.M_mm.D_mc.M_mq.D_mf.M_ms.D_mj.M_mw.D_ml.M_my.D_aOp.M_aLo.D_lV')
                for element in time_elements:
                    time_text = element.text.strip()
                    if time_text and re.search(r'\d+\s*(小時|天|分鐘|月|年)前', time_text):
                        post_time = time_text
                        break
            except:
                pass
        
        # 方法3: 从文本中提取时间
        if not post_time:
            time_match = re.search(r'(\d+\s*(小時|天|分鐘|月|年)前)', container_text)
            if time_match:
                post_time = time_match.group(1)
        
        # 获取商品链接
        link = ""
        try:
            if container.tag_name == "a":
                link = container.get_attribute("href")
            else:
                link_elements = container.find_elements(By.TAG_NAME, "a")
                for link_element in link_elements:
                    href = link_element.get_attribute("href")
                    if href and "/p/" in href:
                        link = href
                        break
        except:
            pass
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        print(f"从容器提取商品信息时出错: {e}")
        return None

def extract_products_by_structure(driver, seen_products):
    """通过页面结构分析提取商品信息"""
    products = []
    
    try:
        # 获取页面所有p元素进行分析
        all_p_elements = driver.find_elements(By.TAG_NAME, "p")
        print(f"页面共有 {len(all_p_elements)} 个p元素")
        
        # 分析包含Labubu的p元素
        labubu_elements = []
        for p_element in all_p_elements:
            try:
                text = p_element.text.strip()
                if text and 'labubu' in text.lower():
                    labubu_elements.append(p_element)
            except:
                continue
        
        print(f"找到 {len(labubu_elements)} 个包含Labubu的p元素")
        
        for p_element in labubu_elements:  # 处理前10个
            try:
                product_info = analyze_element_structure(p_element)
                if product_info and product_info['title']:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ 结构分析提取: {product_info['title'][:30]}... | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
            except:
                continue
                
    except Exception as e:
        print(f"结构分析提取时出错: {e}")
    
    return products

def analyze_element_structure(element):
    """分析元素结构来提取商品信息"""
    try:
        title = element.text.strip()
        
        # 向上查找父容器
        container = element
        for i in range(5):
            container = container.find_element(By.XPATH, "./..")
            container_class = container.get_attribute('class') or ''
            if container_class:
                break
        
        container_text = container.text
        
        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"
        
        # 在容器中查找卖家和时间
        seller = ""
        post_time = ""
        
        # 查找所有兄弟p元素
        try:
            parent = element.find_element(By.XPATH, "./..")
            sibling_p_elements = parent.find_elements(By.TAG_NAME, "p")
            
            for sibling in sibling_p_elements:
                sibling_text = sibling.text.strip()
                if sibling_text and sibling_text != title:
                    # 检查是否是卖家
                    sibling_class = sibling.get_attribute('class') or ''
                    if 'D_mm' in sibling_class and 'M_mz' in sibling_class:
                        seller = sibling_text
                    # 检查是否是时间
                    elif 'D_aOp' in sibling_class or 'color:#00bfa2' in sibling.get_attribute('style') or '':
                        if re.search(r'\d+\s*(小時|天|分鐘|月|年)前', sibling_text):
                            post_time = sibling_text
        except:
            pass
        
        # 获取链接
        link = ""
        try:
            link_element = container.find_element(By.TAG_NAME, "a")
            link = link_element.get_attribute("href")
        except:
            pass
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        return None

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info['seller']}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

# 找到整个 click_show_more_button 函数（大约在V3代码的第400行左右）
# 用这个改进版本替换整个函数：

def click_show_more_button(driver, wait):
    """改进的翻页按钮点击"""
    try:
        # 方法1: 通过按钮文本
        show_more_buttons = driver.find_elements(By.XPATH, 
            "//button[contains(text(), '顯示更多結果') or contains(text(), 'Show more results')]"
        )
        
        for button in show_more_buttons:
            try:
                if button.is_displayed() and button.is_enabled():
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                    time.sleep(2)  # 增加等待时间
                    driver.execute_script("arguments[0].click();", button)
                    print("成功点击翻页按钮")
                    return True
            except:
                continue
        
        # 方法2: 通过JavaScript直接调用可能的事件
        try:
            driver.execute_script("""
                var buttons = document.querySelectorAll('button');
                for (var i = 0; i < buttons.length; i++) {
                    if (buttons[i].textContent.includes('顯示更多結果') || 
                        buttons[i].textContent.includes('Show more results')) {
                        buttons[i].click();
                        return true;
                    }
                }
                return false;
            """)
            print("通过JavaScript点击翻页按钮")
            return True
        except:
            pass
            
        print("未找到可点击的翻页按钮")
        return False
        
    except Exception as e:
        print(f"点击翻页按钮时出错: {e}")
        return False

def save_data_to_desktop(products):
    """保存数据到桌面，使用固定文件名"""
    if not products:
        print("没有数据可保存")
        return None
    
    print(f"准备保存 {len(products)} 个商品数据到桌面")
    
    try:
        # 获取桌面路径
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = "carousell_labubu_products.csv"
        filepath = os.path.join(desktop_path, filename)
        
        # 创建DataFrame
        df = pd.DataFrame(products)
        
        # 重新排列列的顺序
        column_order = ['title', 'price', 'seller', 'post_time', 'link', 'crawl_time']
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]
        
        # 重命名列名为中文
        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户', 
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        df = df.rename(columns=column_mapping)
        
        # 检查文件是否已存在
        if os.path.exists(filepath):
            # 读取现有数据
            existing_df = pd.read_csv(filepath, encoding='utf-8-sig')
            # 合并数据并去重
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            # 去重（基于商品名称、价格、发布用户）
            combined_df = combined_df.drop_duplicates(subset=['商品名称', '价格', '发布用户'], keep='last')
            # 按爬取时间排序
            combined_df = combined_df.sort_values('爬取时间', ascending=False)
            print(f"合并数据: 原有 {len(existing_df)} 条，新增 {len(df)} 条，去重后 {len(combined_df)} 条")
        else:
            combined_df = df
            print("创建新文件")
        
        # 保存到桌面
        combined_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 数据已保存到桌面: {filepath}")
        print(f"📊 总数据量: {len(combined_df)} 条记录")
        
        # 显示数据完整性统计
        display_data_statistics(combined_df)
        
        return filepath
        
    except Exception as e:
        print(f"保存数据到桌面失败: {e}")
        return None

def display_data_statistics(df):
    """显示数据完整性统计"""
    print("\n" + "="*50)
    print("📈 数据完整性统计:")
    print("="*50)
    print(f"总记录数: {len(df)}")
    print(f"有商品名称: {df['商品名称'].notna().sum()}")
    print(f"有价格信息: {df['价格'].notna().sum()}")
    print(f"有发布用户: {df['发布用户'].notna().sum()}")
    print(f"有发布时间: {df['发布时间'].notna().sum()}")
    print(f"有商品链接: {df['商品链接'].notna().sum()}")

def check_desktop_file():
    """检查桌面数据文件"""
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    filepath = os.path.join(desktop_path, "carousell_labubu_products.csv")
    
    if os.path.exists(filepath):
        df = pd.read_csv(filepath, encoding='utf-8-sig')
        print(f"📁 桌面文件: {filepath}")
        print(f"📊 现有数据量: {len(df)} 条记录")
        
        # 显示前5条数据的详细信息
        print("\n📋 前5条数据详情:")
        for i, row in df.head().iterrows():
            print(f"{i+1}. 商品: {row['商品名称'][:40]}...")
            print(f"   价格: {row['价格']}")
            print(f"   卖家: {row['发布用户']}")
            print(f"   时间: {row['发布时间']}")
            print()
        
        return True
    else:
        print("❌ 桌面未找到数据文件")
        return False

# 测试执行
if __name__ == "__main__":
    print("🚀 开始爬取Carousell Labubu商品...")
    
    # 先检查桌面文件
    print("\n" + "="*60)
    print("🔍 检查桌面数据文件:")
    print("="*60)
    check_desktop_file()
    
    # 开始爬取
    products, saved_file = scrape_carousell_labubu()
    
    if products and saved_file:
        print(f"\n✅ 爬取完成！成功获取 {len(products)} 个新商品")
        print(f"💾 所有数据已保存到: {saved_file}")
    elif products:
        print(f"\n⚠️  爬取到 {len(products)} 个商品，但文件保存失败")
    else:
        print("❌ 爬取失败")
    
    # 最后再次检查桌面文件
    print("\n" + "="*60)
    print("📋 最终文件检查:")
    print("="*60)
    check_desktop_file()
    

🚀 开始爬取Carousell Labubu商品...

🔍 检查桌面数据文件:
❌ 桌面未找到数据文件
正在访问搜索页面: https://www.carousell.com.hk/search/labubu?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search
页面body加载完成

=== 正在处理第 1 页 ===
开始滚动页面...
滚动 1/5 次
滚动 2/5 次
页面高度未变化，停止滚动
🔍 找到 96 个商品容器
✅ [1/96] 成功提取: kaka158936... | 卖家: kaka158936 | 时间: 4 分鐘前
❌ [2/96] 无效商品
✅ [3/96] 成功提取: yume_hui... | 卖家: yume_hui | 时间: 6 分鐘前
✅ [4/96] 成功提取: 買家保障... | 卖家: 全新 | 时间: 
✅ [5/96] 成功提取: ivin130.1... | 卖家: ivin130.1 | 时间: 12 分鐘前
✅ [6/96] 成功提取: Labubu 4.0 迷你 Pin For Love 盲盒 字母... | 卖家: 全新 | 时间: 
✅ [7/96] 成功提取: ahanh... | 卖家: ahanh | 时间: 33 分鐘前
✅ [8/96] 成功提取: 買家保障... | 卖家: 全新 | 时间: 
✅ [9/96] 成功提取: lalarsing... | 卖家: lalarsing | 时间: 36 分鐘前
✅ [10/96] 成功提取: 💢⭐️全新現貨⭐️💢 泡泡瑪特 Labubu 心底密碼 字母掛件 英文 字母掛件... | 卖家: 全新 | 时间: 
✅ [11/96] 成功提取: buzzlazy... | 卖家: buzzlazy | 时间: 38 分鐘前
✅ [12/96] 成功提取: 買家保障... | 卖家: 全新 | 时间: 
✅ [13/96] 成功提取: tsangjaijai... | 卖家: tsan

In [20]:
#pucky

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime
import re
import hashlib
import glob

def scrape_carousell_pucky():
    chrome_options = Options()
    # 先去掉无头模式进行调试
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    all_products = []
    max_pages = 10  # 增加到10页
    seen_products = set()
    
    try:
        # 使用提供的Pucky搜索链接
        search_url = "https://www.carousell.com.hk/search/pucky?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search"
        print(f"正在访问Pucky搜索页面: {search_url}")
        driver.get(search_url)
        
        # 使用显式等待
        wait = WebDriverWait(driver, 15)
        
        # 等待页面加载
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        print("页面body加载完成")
        
        current_page = 1
        
        while current_page <= max_pages:
            print(f"\n=== 正在处理第 {current_page} 页 ===")
            
            # 滚动页面以加载所有内容
            scroll_page(driver)
            
            # 等待新内容加载
            time.sleep(2)
            
            # 提取当前页面商品信息
            products = extract_products_new_method(driver, seen_products)
            if products:
                all_products.extend(products)
                print(f"第 {current_page} 页找到 {len(products)} 个新商品")
                print(f"累计找到 {len(all_products)} 个唯一商品")
            else:
                print(f"第 {current_page} 页未找到新商品")
            
            # 检查是否还有更多内容
            if not click_show_more_button(driver, wait):
                print("没有找到'顯示更多結果'按钮，可能已到最后一页")
                break
            
            current_page += 1
            time.sleep(3)
            
        print(f"\n爬取完成！总共找到 {len(all_products)} 个唯一商品")
        
        # 保存数据到桌面
        saved_file = save_data_to_desktop(all_products, "pucky")
        
        return all_products, saved_file
        
    except Exception as e:
        print(f"爬取过程中出错: {e}")
        # 即使出错也尝试保存已收集的数据
        if all_products:
            saved_file = save_data_to_desktop(all_products, "pucky")
            return all_products, saved_file
        return None, None
    finally:
        print("关闭浏览器...")
        driver.quit()

def scroll_page(driver):
    """改进的页面滚动"""
    print("开始滚动页面...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 增加滚动次数和间隔
    for i in range(5):  # 增加到5次
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"滚动 {i+1}/5 次")
        time.sleep(2)  # 增加到2秒
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("页面高度未变化，停止滚动")
            break
        last_height = new_height
    
    # 额外向上滚动一点触发懒加载
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 500);")
    time.sleep(1)

def extract_products_new_method(driver, seen_products):
    """新的商品信息提取方法 - 直接从商品卡片结构提取"""
    products = []
    
    try:
        # 方法1: 通过商品卡片容器查找
        product_containers = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid*="listing-card"], div[class*="D_pA"], div[class*="D_qh"], div[class*="D_qW"], a[href*="/p/"]')
        
        print(f"🔍 找到 {len(product_containers)} 个商品容器")
        
        valid_count = 0
        for i, container in enumerate(product_containers):  # 处理所有容器，不限制数量
            try:
                product_info = extract_product_from_container(container)
                if product_info and product_info['title']:
                    # 创建商品唯一标识
                    product_id = create_product_id(product_info)
                    
                    # 检查是否重复
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        valid_count += 1
                        print(f"✅ [{i+1}/{len(product_containers)}] 成功提取: {product_info['title'][:40]}... | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
                else:
                    print(f"❌ [{i+1}/{len(product_containers)}] 无效商品")
            except Exception as e:
                print(f"⚠️ [{i+1}/{len(product_containers)}] 处理失败: {e}")
                continue
        
        print(f"📊 有效商品: {valid_count}/{len(product_containers)}")
        
        # 如果方法1没找到，使用方法2
        if not products:
            products = extract_products_by_structure(driver, seen_products)
            
    except Exception as e:
        print(f"提取商品时出错: {e}")
    
    return products

def extract_product_from_container(container):
    """从商品容器提取完整信息"""
    try:
        # 获取容器内所有文本
        container_text = container.text.strip()
        if not container_text or len(container_text) < 10:  # 降低长度限制
            return None
        
        # 提取商品名称 - 查找标题元素
        title = ""
        title_selectors = [
            'p.D_lX.D_lY.D_mc.D_mg.D_mj.D_ml.D_mh.D_lU',
            'p[class*="D_lX"]',
            'h3', 'h4', 'p', 'span'  # 添加更多标签
        ]
        
        for selector in title_selectors:
            try:
                title_elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in title_elements:
                    text = element.text.strip()
                    # 放宽条件：不强制要求包含pucky
                    if text and len(text) > 2:
                        title = text
                        break
                if title:
                    break
            except:
                continue
        
        if not title:
            # 从文本中提取可能的标题
            lines = container_text.split('\n')
            for line in lines:
                if len(line) > 3:  # 放宽条件
                    title = line
                    break
        
        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"
        
        # 提取发布用户
        seller = ""
        # 方法1: 通过data-testid查找
        try:
            seller_elements = container.find_elements(By.CSS_SELECTOR, '[data-testid="listing-card-text-seller-name"]')
            for element in seller_elements:
                seller_text = element.text.strip()
                if seller_text:
                    seller = seller_text
                    break
        except:
            pass
        
        # 方法2: 查找特定类名的元素
        if not seller:
            try:
                seller_elements = container.find_elements(By.CSS_SELECTOR, 'p.D_lX.M_ml.D_lY.M_mm.D_mc.M_mq.D_mf.M_ms.D_mj.M_mw.D_mm.M_mz.D_lU')
                for element in seller_elements:
                    seller_text = element.text.strip()
                    if seller_text:
                        seller = seller_text
                        break
            except:
                pass
        
        # 方法3: 从文本中分析可能的用户名
        if not seller:
            lines = container_text.split('\n')
            for line in lines:
                line = line.strip()
                if (line and 
                    not re.search(r'(HK\$|\$|\d+)', line) and
                    not re.search(r'\d+\s*(小時|天|分鐘|月|年)前', line) and
                    line != title and
                    len(line) > 1 and len(line) < 25 and
                    not any(keyword in line.lower() for keyword in ['pucky', 'like', '留言', '立即聊聊'])):
                    seller = line
                    break
        
        # 提取发布时间
        post_time = ""
        # 方法1: 通过特定样式查找
        try:
            time_elements = container.find_elements(By.CSS_SELECTOR, 'p[style*="color:#00bfa2"]')
            for element in time_elements:
                time_text = element.text.strip()
                if time_text and re.search(r'\d+\s*(小時|天|分鐘|月|年)前', time_text):
                    post_time = time_text
                    break
        except:
            pass
        
        # 方法2: 查找特定类名的元素
        if not post_time:
            try:
                time_elements = container.find_elements(By.CSS_SELECTOR, 'p.D_lX.M_ml.D_lY.M_mm.D_mc.M_mq.D_mf.M_ms.D_mj.M_mw.D_ml.M_my.D_aOp.M_aLo.D_lV')
                for element in time_elements:
                    time_text = element.text.strip()
                    if time_text and re.search(r'\d+\s*(小時|天|分鐘|月|年)前', time_text):
                        post_time = time_text
                        break
            except:
                pass
        
        # 方法3: 从文本中提取时间
        if not post_time:
            time_match = re.search(r'(\d+\s*(小時|天|分鐘|月|年)前)', container_text)
            if time_match:
                post_time = time_match.group(1)
        
        # 获取商品链接
        link = ""
        try:
            if container.tag_name == "a":
                link = container.get_attribute("href")
            else:
                link_elements = container.find_elements(By.TAG_NAME, "a")
                for link_element in link_elements:
                    href = link_element.get_attribute("href")
                    if href and "/p/" in href:
                        link = href
                        break
        except:
            pass
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        print(f"从容器提取商品信息时出错: {e}")
        return None

def extract_products_by_structure(driver, seen_products):
    """通过页面结构分析提取商品信息"""
    products = []
    
    try:
        # 获取页面所有p元素进行分析
        all_p_elements = driver.find_elements(By.TAG_NAME, "p")
        print(f"页面共有 {len(all_p_elements)} 个p元素")
        
        # 分析包含Pucky的p元素
        pucky_elements = []
        for p_element in all_p_elements:
            try:
                text = p_element.text.strip()
                if text and 'pucky' in text.lower():
                    pucky_elements.append(p_element)
            except:
                continue
        
        print(f"找到 {len(pucky_elements)} 个包含Pucky的p元素")
        
        for p_element in pucky_elements:  # 处理所有找到的元素
            try:
                product_info = analyze_element_structure(p_element)
                if product_info and product_info['title']:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ 结构分析提取: {product_info['title'][:30]}... | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
            except:
                continue
                
    except Exception as e:
        print(f"结构分析提取时出错: {e}")
    
    return products

def analyze_element_structure(element):
    """分析元素结构来提取商品信息"""
    try:
        title = element.text.strip()
        
        # 向上查找父容器
        container = element
        for i in range(5):
            container = container.find_element(By.XPATH, "./..")
            container_class = container.get_attribute('class') or ''
            if container_class:
                break
        
        container_text = container.text
        
        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"
        
        # 在容器中查找卖家和时间
        seller = ""
        post_time = ""
        
        # 查找所有兄弟p元素
        try:
            parent = element.find_element(By.XPATH, "./..")
            sibling_p_elements = parent.find_elements(By.TAG_NAME, "p")
            
            for sibling in sibling_p_elements:
                sibling_text = sibling.text.strip()
                if sibling_text and sibling_text != title:
                    # 检查是否是卖家
                    sibling_class = sibling.get_attribute('class') or ''
                    if 'D_mm' in sibling_class and 'M_mz' in sibling_class:
                        seller = sibling_text
                    # 检查是否是时间
                    elif 'D_aOp' in sibling_class or 'color:#00bfa2' in sibling.get_attribute('style') or '':
                        if re.search(r'\d+\s*(小時|天|分鐘|月|年)前', sibling_text):
                            post_time = sibling_text
        except:
            pass
        
        # 获取链接
        link = ""
        try:
            link_element = container.find_element(By.TAG_NAME, "a")
            link = link_element.get_attribute("href")
        except:
            pass
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        return None

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info['seller']}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def click_show_more_button(driver, wait):
    """点击'顯示更多結果'按钮"""
    try:
        # 方法1: 通过按钮文本内容查找
        show_more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), '顯示更多結果')]")
        
        for button in show_more_buttons:
            if button.is_displayed() and button.is_enabled():
                # 滚动到按钮位置
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                time.sleep(1)
                
                # 点击按钮
                driver.execute_script("arguments[0].click();", button)
                print("成功点击'顯示更多結果'按钮")
                return True
        
        print("未找到可点击的'顯示更多結果'按钮")
        return False
        
    except Exception as e:
        print(f"点击'顯示更多結果'按钮时出错: {e}")
        return False

def save_data_to_desktop(products, product_type="pucky"):
    """保存数据到桌面，使用固定文件名"""
    if not products:
        print("没有数据可保存")
        return None
    
    print(f"准备保存 {len(products)} 个商品数据到桌面")
    
    try:
        # 获取桌面路径
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = f"carousell_{product_type}_products.csv"
        filepath = os.path.join(desktop_path, filename)
        
        # 创建DataFrame
        df = pd.DataFrame(products)
        
        # 重新排列列的顺序
        column_order = ['title', 'price', 'seller', 'post_time', 'link', 'crawl_time']
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]
        
        # 重命名列名为中文
        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户', 
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        df = df.rename(columns=column_mapping)
        
        # 检查文件是否已存在
        if os.path.exists(filepath):
            # 读取现有数据
            existing_df = pd.read_csv(filepath, encoding='utf-8-sig')
            # 合并数据并去重
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            # 去重（基于商品名称、价格、发布用户）
            combined_df = combined_df.drop_duplicates(subset=['商品名称', '价格', '发布用户'], keep='last')
            # 按爬取时间排序
            combined_df = combined_df.sort_values('爬取时间', ascending=False)
            print(f"合并数据: 原有 {len(existing_df)} 条，新增 {len(df)} 条，去重后 {len(combined_df)} 条")
        else:
            combined_df = df
            print("创建新文件")
        
        # 保存到桌面
        combined_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 数据已保存到桌面: {filepath}")
        print(f"📊 总数据量: {len(combined_df)} 条记录")
        
        # 显示数据完整性统计
        display_data_statistics(combined_df)
        
        return filepath
        
    except Exception as e:
        print(f"保存数据到桌面失败: {e}")
        return None

def display_data_statistics(df):
    """显示数据完整性统计"""
    print("\n" + "="*50)
    print("📈 数据完整性统计:")
    print("="*50)
    print(f"总记录数: {len(df)}")
    print(f"有商品名称: {df['商品名称'].notna().sum()}")
    print(f"有价格信息: {df['价格'].notna().sum()}")
    print(f"有发布用户: {df['发布用户'].notna().sum()}")
    print(f"有发布时间: {df['发布时间'].notna().sum()}")
    print(f"有商品链接: {df['商品链接'].notna().sum()}")

def check_desktop_file(product_type="pucky"):
    """检查桌面数据文件"""
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    filename = f"carousell_{product_type}_products.csv"
    filepath = os.path.join(desktop_path, filename)
    
    if os.path.exists(filepath):
        df = pd.read_csv(filepath, encoding='utf-8-sig')
        print(f"📁 桌面文件: {filepath}")
        print(f"📊 现有数据量: {len(df)} 条记录")
        
        # 显示前5条数据的详细信息
        print("\n📋 前5条数据详情:")
        for i, row in df.head().iterrows():
            print(f"{i+1}. 商品: {row['商品名称'][:40]}...")
            print(f"   价格: {row['价格']}")
            print(f"   卖家: {row['发布用户']}")
            print(f"   时间: {row['发布时间']}")
            print()
        
        return True
    else:
        print(f"❌ 桌面未找到{product_type}数据文件")
        return False

# 测试执行
if __name__ == "__main__":
    print("🚀 开始爬取Carousell Pucky商品...")
    
    # 先检查桌面文件
    print("\n" + "="*60)
    print("🔍 检查桌面数据文件:")
    print("="*60)
    check_desktop_file("pucky")
    
    # 开始爬取
    products, saved_file = scrape_carousell_pucky()
    
    if products and saved_file:
        print(f"\n✅ 爬取完成！成功获取 {len(products)} 个新商品")
        print(f"💾 所有数据已保存到: {saved_file}")
    elif products:
        print(f"\n⚠️  爬取到 {len(products)} 个商品，但文件保存失败")
    else:
        print("❌ 爬取失败")
    
    # 最后再次检查桌面文件
    print("\n" + "="*60)
    print("📋 最终文件检查:")
    print("="*60)
    check_desktop_file("pucky")

🚀 开始爬取Carousell Pucky商品...

🔍 检查桌面数据文件:
❌ 桌面未找到pucky数据文件
正在访问Pucky搜索页面: https://www.carousell.com.hk/search/pucky?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search
页面body加载完成

=== 正在处理第 1 页 ===
开始滚动页面...
滚动 1/5 次
滚动 2/5 次
页面高度未变化，停止滚动
🔍 找到 94 个商品容器
✅ [1/94] 成功提取: zhuge_shendian... | 卖家: zhuge_shendian | 时间: 31 分鐘前
✅ [2/94] 成功提取: 《現貨》全新未開盒/拆盒未拆袋 Pucky豆子精靈飛行系列 掛件盲盒 100%正... | 卖家: 全新 | 时间: 
✅ [3/94] 成功提取: twiggy.l... | 卖家: twiggy.l | 时间: 11 小時前
✅ [4/94] 成功提取: 買家保障... | 卖家: 全新 | 时间: 
✅ [5/94] 成功提取: regincat123... | 卖家: regincat123 | 时间: 2 天前
✅ [6/94] 成功提取: [買/換］Pucky Egg Beanie 蛋蛋豆子系列 / 蛋撻... | 卖家: 全新 | 时间: 
✅ [7/94] 成功提取: macy316... | 卖家: macy316 | 时间: 2 天前
✅ [8/94] 成功提取: 買家保障... | 卖家: 全新 | 时间: 
✅ [9/94] 成功提取: weirdo232... | 卖家: weirdo232 | 时间: 3 天前
✅ [10/94] 成功提取: 買家保障... | 卖家: 全新 | 时间: 
✅ [11/94] 成功提取: rear.rear... | 卖家: rear.rear | 时间: 3 天前
✅ [12/94] 成功提取: 買家保障... | 卖家: 幾乎全新 

In [18]:
#sweetbean

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime
import re
import hashlib
import glob

def scrape_carousell_sweetbean():
    chrome_options = Options()
    # 先去掉无头模式进行调试
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    all_products = []
    max_pages = 10  # 增加到10页
    seen_products = set()
    
    try:
        # 使用提供的Sweetbean搜索链接
        search_url = "https://www.carousell.com.hk/search/sweetbean?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search"
        print(f"正在访问Sweetbean搜索页面: {search_url}")
        driver.get(search_url)
        
        # 使用显式等待
        wait = WebDriverWait(driver, 15)
        
        # 等待页面加载
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        print("页面body加载完成")
        
        current_page = 1
        
        while current_page <= max_pages:
            print(f"\n=== 正在处理第 {current_page} 页 ===")
            
            # 滚动页面以加载所有内容
            scroll_page(driver)
            
            # 等待新内容加载
            time.sleep(2)
            
            # 提取当前页面商品信息
            products = extract_products_new_method(driver, seen_products)
            if products:
                all_products.extend(products)
                print(f"第 {current_page} 页找到 {len(products)} 个新商品")
                print(f"累计找到 {len(all_products)} 个唯一商品")
            else:
                print(f"第 {current_page} 页未找到新商品")
            
            # 检查是否还有更多内容
            if not click_show_more_button(driver, wait):
                print("没有找到'顯示更多結果'按钮，可能已到最后一页")
                break
            
            current_page += 1
            time.sleep(3)
            
        print(f"\n爬取完成！总共找到 {len(all_products)} 个唯一商品")
        
        # 保存数据到桌面
        saved_file = save_data_to_desktop(all_products, "sweetbean")
        
        return all_products, saved_file
        
    except Exception as e:
        print(f"爬取过程中出错: {e}")
        # 即使出错也尝试保存已收集的数据
        if all_products:
            saved_file = save_data_to_desktop(all_products, "sweetbean")
            return all_products, saved_file
        return None, None
    finally:
        print("关闭浏览器...")
        driver.quit()

def scroll_page(driver):
    """改进的页面滚动"""
    print("开始滚动页面...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 增加滚动次数和间隔
    for i in range(5):  # 增加到5次
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"滚动 {i+1}/5 次")
        time.sleep(2)  # 增加到2秒
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("页面高度未变化，停止滚动")
            break
        last_height = new_height
    
    # 额外向上滚动一点触发懒加载
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 500);")
    time.sleep(1)

def extract_products_new_method(driver, seen_products):
    """新的商品信息提取方法 - 直接从商品卡片结构提取"""
    products = []
    
    try:
        # 方法1: 通过商品卡片容器查找
        product_containers = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid*="listing-card"], div[class*="D_pA"], div[class*="D_qh"], div[class*="D_qW"], a[href*="/p/"]')
        
        print(f"🔍 找到 {len(product_containers)} 个商品容器")
        
        valid_count = 0
        for i, container in enumerate(product_containers):  # 处理所有容器，不限制数量
            try:
                product_info = extract_product_from_container(container)
                if product_info and product_info['title']:
                    # 创建商品唯一标识
                    product_id = create_product_id(product_info)
                    
                    # 检查是否重复
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        valid_count += 1
                        print(f"✅ [{i+1}/{len(product_containers)}] 成功提取: {product_info['title'][:40]}... | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
                else:
                    print(f"❌ [{i+1}/{len(product_containers)}] 无效商品")
            except Exception as e:
                print(f"⚠️ [{i+1}/{len(product_containers)}] 处理失败: {e}")
                continue
        
        print(f"📊 有效商品: {valid_count}/{len(product_containers)}")
        
        # 如果方法1没找到，使用方法2
        if not products:
            products = extract_products_by_structure(driver, seen_products)
            
    except Exception as e:
        print(f"提取商品时出错: {e}")
    
    return products

def extract_product_from_container(container):
    """从商品容器提取完整信息"""
    try:
        # 获取容器内所有文本
        container_text = container.text.strip()
        if not container_text or len(container_text) < 10:  # 降低长度限制
            return None
        
        # 提取商品名称 - 查找标题元素
        title = ""
        title_selectors = [
            'p.D_lX.D_lY.D_mc.D_mg.D_mj.D_ml.D_mh.D_lU',
            'p[class*="D_lX"]',
            'h3', 'h4', 'p', 'span'  # 添加更多标签
        ]
        
        for selector in title_selectors:
            try:
                title_elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in title_elements:
                    text = element.text.strip()
                    # 放宽条件：不强制要求包含sweetbean
                    if text and len(text) > 2:
                        title = text
                        break
                if title:
                    break
            except:
                continue
        
        if not title:
            # 从文本中提取可能的标题
            lines = container_text.split('\n')
            for line in lines:
                if len(line) > 3:  # 放宽条件
                    title = line
                    break
        
        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"
        
        # 提取发布用户
        seller = ""
        # 方法1: 通过data-testid查找
        try:
            seller_elements = container.find_elements(By.CSS_SELECTOR, '[data-testid="listing-card-text-seller-name"]')
            for element in seller_elements:
                seller_text = element.text.strip()
                if seller_text:
                    seller = seller_text
                    break
        except:
            pass
        
        # 方法2: 查找特定类名的元素
        if not seller:
            try:
                seller_elements = container.find_elements(By.CSS_SELECTOR, 'p.D_lX.M_ml.D_lY.M_mm.D_mc.M_mq.D_mf.M_ms.D_mj.M_mw.D_mm.M_mz.D_lU')
                for element in seller_elements:
                    seller_text = element.text.strip()
                    if seller_text:
                        seller = seller_text
                        break
            except:
                pass
        
        # 方法3: 从文本中分析可能的用户名
        if not seller:
            lines = container_text.split('\n')
            for line in lines:
                line = line.strip()
                if (line and 
                    not re.search(r'(HK\$|\$|\d+)', line) and
                    not re.search(r'\d+\s*(小時|天|分鐘|月|年)前', line) and
                    line != title and
                    len(line) > 1 and len(line) < 25 and
                    not any(keyword in line.lower() for keyword in ['sweetbean', 'like', '留言', '立即聊聊'])):
                    seller = line
                    break
        
        # 提取发布时间
        post_time = ""
        # 方法1: 通过特定样式查找
        try:
            time_elements = container.find_elements(By.CSS_SELECTOR, 'p[style*="color:#00bfa2"]')
            for element in time_elements:
                time_text = element.text.strip()
                if time_text and re.search(r'\d+\s*(小時|天|分鐘|月|年)前', time_text):
                    post_time = time_text
                    break
        except:
            pass
        
        # 方法2: 查找特定类名的元素
        if not post_time:
            try:
                time_elements = container.find_elements(By.CSS_SELECTOR, 'p.D_lX.M_ml.D_lY.M_mm.D_mc.M_mq.D_mf.M_ms.D_mj.M_mw.D_ml.M_my.D_aOp.M_aLo.D_lV')
                for element in time_elements:
                    time_text = element.text.strip()
                    if time_text and re.search(r'\d+\s*(小時|天|分鐘|月|年)前', time_text):
                        post_time = time_text
                        break
            except:
                pass
        
        # 方法3: 从文本中提取时间
        if not post_time:
            time_match = re.search(r'(\d+\s*(小時|天|分鐘|月|年)前)', container_text)
            if time_match:
                post_time = time_match.group(1)
        
        # 获取商品链接
        link = ""
        try:
            if container.tag_name == "a":
                link = container.get_attribute("href")
            else:
                link_elements = container.find_elements(By.TAG_NAME, "a")
                for link_element in link_elements:
                    href = link_element.get_attribute("href")
                    if href and "/p/" in href:
                        link = href
                        break
        except:
            pass
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        print(f"从容器提取商品信息时出错: {e}")
        return None

def extract_products_by_structure(driver, seen_products):
    """通过页面结构分析提取商品信息"""
    products = []
    
    try:
        # 获取页面所有p元素进行分析
        all_p_elements = driver.find_elements(By.TAG_NAME, "p")
        print(f"页面共有 {len(all_p_elements)} 个p元素")
        
        # 分析包含Sweetbean的p元素
        sweetbean_elements = []
        for p_element in all_p_elements:
            try:
                text = p_element.text.strip()
                if text and 'sweetbean' in text.lower():
                    sweetbean_elements.append(p_element)
            except:
                continue
        
        print(f"找到 {len(sweetbean_elements)} 个包含Sweetbean的p元素")
        
        for p_element in sweetbean_elements:  # 处理所有找到的元素
            try:
                product_info = analyze_element_structure(p_element)
                if product_info and product_info['title']:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ 结构分析提取: {product_info['title'][:30]}... | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
            except:
                continue
                
    except Exception as e:
        print(f"结构分析提取时出错: {e}")
    
    return products

def analyze_element_structure(element):
    """分析元素结构来提取商品信息"""
    try:
        title = element.text.strip()
        
        # 向上查找父容器
        container = element
        for i in range(5):
            container = container.find_element(By.XPATH, "./..")
            container_class = container.get_attribute('class') or ''
            if container_class:
                break
        
        container_text = container.text
        
        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"
        
        # 在容器中查找卖家和时间
        seller = ""
        post_time = ""
        
        # 查找所有兄弟p元素
        try:
            parent = element.find_element(By.XPATH, "./..")
            sibling_p_elements = parent.find_elements(By.TAG_NAME, "p")
            
            for sibling in sibling_p_elements:
                sibling_text = sibling.text.strip()
                if sibling_text and sibling_text != title:
                    # 检查是否是卖家
                    sibling_class = sibling.get_attribute('class') or ''
                    if 'D_mm' in sibling_class and 'M_mz' in sibling_class:
                        seller = sibling_text
                    # 检查是否是时间
                    elif 'D_aOp' in sibling_class or 'color:#00bfa2' in sibling.get_attribute('style') or '':
                        if re.search(r'\d+\s*(小時|天|分鐘|月|年)前', sibling_text):
                            post_time = sibling_text
        except:
            pass
        
        # 获取链接
        link = ""
        try:
            link_element = container.find_element(By.TAG_NAME, "a")
            link = link_element.get_attribute("href")
        except:
            pass
        
        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return product_info
        
    except Exception as e:
        return None

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info['seller']}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def click_show_more_button(driver, wait):
    """点击'顯示更多結果'按钮"""
    try:
        # 方法1: 通过按钮文本内容查找
        show_more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), '顯示更多結果')]")
        
        for button in show_more_buttons:
            if button.is_displayed() and button.is_enabled():
                # 滚动到按钮位置
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                time.sleep(1)
                
                # 点击按钮
                driver.execute_script("arguments[0].click();", button)
                print("成功点击'顯示更多結果'按钮")
                return True
        
        print("未找到可点击的'顯示更多結果'按钮")
        return False
        
    except Exception as e:
        print(f"点击'顯示更多結果'按钮时出错: {e}")
        return False

def save_data_to_desktop(products, product_type="sweetbean"):
    """保存数据到桌面，使用固定文件名"""
    if not products:
        print("没有数据可保存")
        return None
    
    print(f"准备保存 {len(products)} 个商品数据到桌面")
    
    try:
        # 获取桌面路径
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = f"carousell_{product_type}_products.csv"
        filepath = os.path.join(desktop_path, filename)
        
        # 创建DataFrame
        df = pd.DataFrame(products)
        
        # 重新排列列的顺序
        column_order = ['title', 'price', 'seller', 'post_time', 'link', 'crawl_time']
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]
        
        # 重命名列名为中文
        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户', 
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        df = df.rename(columns=column_mapping)
        
        # 检查文件是否已存在
        if os.path.exists(filepath):
            # 读取现有数据
            existing_df = pd.read_csv(filepath, encoding='utf-8-sig')
            # 合并数据并去重
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            # 去重（基于商品名称、价格、发布用户）
            combined_df = combined_df.drop_duplicates(subset=['商品名称', '价格', '发布用户'], keep='last')
            # 按爬取时间排序
            combined_df = combined_df.sort_values('爬取时间', ascending=False)
            print(f"合并数据: 原有 {len(existing_df)} 条，新增 {len(df)} 条，去重后 {len(combined_df)} 条")
        else:
            combined_df = df
            print("创建新文件")
        
        # 保存到桌面
        combined_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 数据已保存到桌面: {filepath}")
        print(f"📊 总数据量: {len(combined_df)} 条记录")
        
        # 显示数据完整性统计
        display_data_statistics(combined_df)
        
        return filepath
        
    except Exception as e:
        print(f"保存数据到桌面失败: {e}")
        return None

def display_data_statistics(df):
    """显示数据完整性统计"""
    print("\n" + "="*50)
    print("📈 数据完整性统计:")
    print("="*50)
    print(f"总记录数: {len(df)}")
    print(f"有商品名称: {df['商品名称'].notna().sum()}")
    print(f"有价格信息: {df['价格'].notna().sum()}")
    print(f"有发布用户: {df['发布用户'].notna().sum()}")
    print(f"有发布时间: {df['发布时间'].notna().sum()}")
    print(f"有商品链接: {df['商品链接'].notna().sum()}")

def check_desktop_file(product_type="sweetbean"):
    """检查桌面数据文件"""
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    filename = f"carousell_{product_type}_products.csv"
    filepath = os.path.join(desktop_path, filename)
    
    if os.path.exists(filepath):
        df = pd.read_csv(filepath, encoding='utf-8-sig')
        print(f"📁 桌面文件: {filepath}")
        print(f"📊 现有数据量: {len(df)} 条记录")
        
        # 显示前5条数据的详细信息
        print("\n📋 前5条数据详情:")
        for i, row in df.head().iterrows():
            print(f"{i+1}. 商品: {row['商品名称'][:40]}...")
            print(f"   价格: {row['价格']}")
            print(f"   卖家: {row['发布用户']}")
            print(f"   时间: {row['发布时间']}")
            print()
        
        return True
    else:
        print(f"❌ 桌面未找到{product_type}数据文件")
        return False

# 测试执行
if __name__ == "__main__":
    print("🚀 开始爬取Carousell Sweetbean商品...")
    
    # 先检查桌面文件
    print("\n" + "="*60)
    print("🔍 检查桌面数据文件:")
    print("="*60)
    check_desktop_file("sweetbean")
    
    # 开始爬取
    products, saved_file = scrape_carousell_sweetbean()
    
    if products and saved_file:
        print(f"\n✅ 爬取完成！成功获取 {len(products)} 个新商品")
        print(f"💾 所有数据已保存到: {saved_file}")
    elif products:
        print(f"\n⚠️  爬取到 {len(products)} 个商品，但文件保存失败")
    else:
        print("❌ 爬取失败")
    
    # 最后再次检查桌面文件
    print("\n" + "="*60)
    print("📋 最终文件检查:")
    print("="*60)
    check_desktop_file("sweetbean")

🚀 开始爬取Carousell Sweetbean商品...

🔍 检查桌面数据文件:
❌ 桌面未找到sweetbean数据文件
正在访问Sweetbean搜索页面: https://www.carousell.com.hk/search/sweetbean?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search
页面body加载完成

=== 正在处理第 1 页 ===
开始滚动页面...
滚动 1/5 次
页面高度未变化，停止滚动
🔍 找到 96 个商品容器
❌ [1/96] 无效商品
❌ [2/96] 无效商品
❌ [3/96] 无效商品
❌ [4/96] 无效商品
❌ [5/96] 无效商品
❌ [6/96] 无效商品
❌ [7/96] 无效商品
❌ [8/96] 无效商品
❌ [9/96] 无效商品
❌ [10/96] 无效商品
❌ [11/96] 无效商品
❌ [12/96] 无效商品
❌ [13/96] 无效商品
❌ [14/96] 无效商品
❌ [15/96] 无效商品
❌ [16/96] 无效商品
❌ [17/96] 无效商品
❌ [18/96] 无效商品
❌ [19/96] 无效商品
❌ [20/96] 无效商品
❌ [21/96] 无效商品
❌ [22/96] 无效商品
❌ [23/96] 无效商品
❌ [24/96] 无效商品
❌ [25/96] 无效商品
❌ [26/96] 无效商品
❌ [27/96] 无效商品
❌ [28/96] 无效商品
❌ [29/96] 无效商品
❌ [30/96] 无效商品
❌ [31/96] 无效商品
❌ [32/96] 无效商品
❌ [33/96] 无效商品
❌ [34/96] 无效商品
❌ [35/96] 无效商品
❌ [36/96] 无效商品
❌ [37/96] 无效商品
❌ [38/96] 无效商品
❌ [39/96] 无效商品
❌ [40/96] 无效商品
❌ [41/96] 无效商品
❌ [42/96] 无效商品
❌ [43/96] 无

In [41]:
#v3 修复商品名称提取错误版本 + 完全修复发布时间提取

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime
import re
import hashlib
import glob

def scrape_carousell_labubu():
    chrome_options = Options()
    # 先去掉无头模式进行调试
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    all_products = []
    max_pages = 3  # 减少页数进行测试
    seen_products = set()

    try:
        # 直接访问搜索页面
        search_url = "https://www.carousell.com.hk/search/labubu?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search"
        print(f"正在访问搜索页面: {search_url}")
        driver.get(search_url)

        # 使用显式等待
        wait = WebDriverWait(driver, 15)

        # 等待页面加载
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        print("页面body加载完成")

        current_page = 1
        while current_page <= max_pages:
            print(f"\n=== 正在处理第 {current_page} 页 ===")

            # 滚动页面以加载所有内容
            scroll_page(driver)

            # 等待新内容加载
            time.sleep(2)

            # 提取当前页面商品信息 - 使用改进的商品名称提取方法
            products = extract_products_new_method(driver, seen_products)
            if products:
                all_products.extend(products)
                print(f"第 {current_page} 页找到 {len(products)} 个新商品")
                print(f"累计找到 {len(all_products)} 个唯一商品")
            else:
                print(f"第 {current_page} 页未找到新商品")

            # 检查是否还有更多内容
            if not click_show_more_button(driver, wait):
                print("没有找到'顯示更多結果'按钮，可能已到最后一页")
                break

            current_page += 1
            time.sleep(3)

        print(f"\n爬取完成！总共找到 {len(all_products)} 个唯一商品")

        # 保存数据到桌面
        saved_file = save_data_to_desktop(all_products)
        return all_products, saved_file

    except Exception as e:
        print(f"爬取过程中出错: {e}")
        # 即使出错也尝试保存已收集的数据
        if all_products:
            saved_file = save_data_to_desktop(all_products)
            return all_products, saved_file
        return None, None
    finally:
        print("关闭浏览器...")
        driver.quit()

def scroll_page(driver):
    """改进的页面滚动"""
    print("开始滚动页面...")
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 增加滚动次数和间隔
    for i in range(5):  # 增加到5次
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"滚动 {i+1}/5 次")
        time.sleep(2)  # 增加到2秒

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("页面高度未变化，停止滚动")
            break
        last_height = new_height

    # 额外向上滚动一点触发懒加载
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 500);")
    time.sleep(1)

def extract_products_new_method(driver, seen_products):
    """新的商品信息提取方法 - 重点修复商品名称提取"""
    products = []
    try:
        # 方法1: 通过商品卡片容器查找（使用更稳定的选择器）
        product_containers = driver.find_elements(By.CSS_SELECTOR,
            'div[data-testid="listing-card"], div[class*="listing-card"], a[href*="/p/"][data-testid*="listing"]'
        )
        print(f"🔍 找到 {len(product_containers)} 个商品容器")

        valid_count = 0
        for i, container in enumerate(product_containers):
            try:
                product_info = extract_product_from_container(container)
                if product_info and product_info['title'] and len(product_info['title']) > 3:
                    # 创建商品唯一标识
                    product_id = create_product_id(product_info)
                    # 检查是否重复
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        valid_count += 1
                        print(f"✅ [{i+1}/{len(product_containers)}] 成功提取: {product_info['title'][:40]}... | 价格: {product_info['price']} | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
                    else:
                        print(f"🔄 [{i+1}/{len(product_containers)}] 重复商品: {product_info['title'][:40]}...")
                else:
                    print(f"❌ [{i+1}/{len(product_containers)}] 无效商品（标题过短或为空）")
            except Exception as e:
                print(f"⚠️ [{i+1}/{len(product_containers)}] 处理失败: {e}")
                continue

        print(f"📊 有效商品: {valid_count}/{len(product_containers)}")

        # 如果方法1没找到，使用方法2
        if not products:
            products = extract_products_by_structure(driver, seen_products)

    except Exception as e:
        print(f"提取商品时出错: {e}")

    return products

def extract_product_from_container(container):
    """从商品容器提取完整信息 - 重点修复发布时间提取"""
    try:
        # 获取容器内所有文本
        container_text = container.text.strip()
        if not container_text or len(container_text) < 20:
            return None

        # --------------------------- 商品标题提取 ---------------------------
        title = extract_title(container, container_text)
        if not title or len(title) < 3:
            print(f"❌ 未找到有效商品标题，容器文本：{container_text[:50]}...")
            return None

        # --------------------------- 价格提取 ---------------------------
        price = extract_price(container_text)

        # --------------------------- 卖家信息提取 ---------------------------
        seller = extract_seller(container, container_text, title)

        # --------------------------- 发布时间提取（完全重写） ---------------------------
        post_time = extract_post_time_comprehensive(container, container_text)
        
        # 获取商品链接
        link = extract_link(container)

        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        return product_info

    except Exception as e:
        print(f"从容器提取商品信息时出错: {e}")
        return None

def extract_title(container, container_text):
    """提取商品标题"""
    title = ""

    # 方法1：通过商品标题专属的data-testid提取（最可靠）
    try:
        title_elements = container.find_elements(By.CSS_SELECTOR, '[data-testid*="title"], [data-testid*="Title"]')
        for element in title_elements:
            text = element.text.strip()
            if text and len(text) > 3:
                title = text
                print(f"📌 通过data-testid找到标题: {title[:30]}...")
                break
    except:
        pass

    # 方法2：通过类名结构查找
    if not title:
        try:
            title_selectors = [
                'p, h1, h2, h3, h4, h5, h6, span, div',
            ]
            for selector in title_selectors:
                title_elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in title_elements:
                    text = element.text.strip()
                    # 标题判断规则
                    if (text and len(text) >= 3 and len(text) <= 100
                        and not re.search(r'(HK\$|\$|\d+)', text)
                        and not re.search(r'\d+\s*(小時|小时|天|分鐘|分钟|月|年)前', text)
                        and not any(keyword in text.lower() for keyword in ['seller', '賣家', 'account', '用戶', 'ID'])
                        and (('labubu' in text.lower()) or any(keyword in text.lower() for keyword in ['盲盒', '公仔', '手办', '全新', '二手', '未拆', '现货']))
                    ):
                        title = text
                        break
                if title:
                    break
        except:
            pass

    # 方法3：从文本中智能提取商品标题
    if not title:
        lines = container_text.split('\n')
        for line in lines:
            line = line.strip()
            if (len(line) >= 3 and len(line) <= 100
                and not re.search(r'(HK\$|\$|\d+,?\d*)', line)
                and not re.search(r'\d+\s*(小時|小时|天|分鐘|分钟|月|年)前', line)
                and not any(keyword in line.lower() for keyword in ['seller', '賣家', 'account', '用戶', 'ID'])
                and (('labubu' in line.lower()) or any(keyword in line.lower() for keyword in ['盲盒', '公仔', '手办', '全新', '二手', '未拆', '现货']))
            ):
                title = line
                break

    return title

def extract_price(container_text):
    """提取价格信息"""
    price = ""
    price_match = re.search(r'(HK\$|\$|HK\s*\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
    if price_match:
        price = f"{price_match.group(1)}{price_match.group(2)}"
    return price

def extract_seller(container, container_text, title):
    """提取卖家信息"""
    seller = ""

    # 方法1: 通过seller专属data-testid查找
    try:
        seller_elements = container.find_elements(By.CSS_SELECTOR, '[data-testid*="seller"], [data-testid*="user"], [data-testid*="author"]')
        for element in seller_elements:
            seller_text = element.text.strip()
            if seller_text and len(seller_text) >= 2 and len(seller_text) <= 20:
                seller = seller_text
                break
    except:
        pass

    # 方法2: 通过seller专属类名查找
    if not seller:
        try:
            seller_elements = container.find_elements(By.CSS_SELECTOR, 'p, span, div')
            for element in seller_elements:
                seller_text = element.text.strip()
                if (seller_text and seller_text != title
                    and len(seller_text) >= 2 and len(seller_text) <= 20
                    and not re.search(r'(HK\$|\$|\d+)', seller_text)
                    and not re.search(r'\d+\s*(小時|小时|天|分鐘|分钟|月|年)前', seller_text)
                    and not any(keyword in seller_text.lower() for keyword in ['labubu', '盲盒', '公仔', '手办', '全新', '二手'])
                ):
                    seller = seller_text
                    break
        except:
            pass

    # 方法3: 从文本中提取卖家名称
    if not seller:
        lines = container_text.split('\n')
        for line in lines:
            line = line.strip()
            if (line and line != title
                and len(line) >= 2 and len(line) <= 20
                and not re.search(r'(HK\$|\$|\d+)', line)
                and not re.search(r'\d+\s*(小時|小时|天|分鐘|分钟|月|年)前', line)
                and not any(keyword in line.lower() for keyword in ['labubu', '盲盒', '公仔', '手办', '全新', '二手'])
            ):
                seller = line
                break

    return seller

def extract_post_time_comprehensive(container, container_text):
    """全面提取发布时间 - 完全重写的逻辑"""
    post_time = ""
    
    # 方法1: 通过时间专属data-testid查找（最可靠）
    try:
        time_elements = container.find_elements(By.CSS_SELECTOR, 
            '[data-testid*="time"], [data-testid*="Time"], [data-testid*="date"], [data-testid*="Date"]'
        )
        for element in time_elements:
            time_text = element.text.strip()
            if time_text and is_valid_time_format(time_text):
                post_time = time_text
                print(f"🕒 通过data-testid找到发布时间: {post_time}")
                break
    except Exception as e:
        print(f"data-testid时间提取失败: {e}")

    # 方法2: 通过时间专属类名查找
    if not post_time:
        try:
            # 扩展时间类名选择器
            time_selectors = [
                '*[class*="time"]',
                '*[class*="Time"]', 
                '*[class*="date"]',
                '*[class*="Date"]',
                '*[class*="posted"]',
                '*[class*="Posted"]',
                '*[class*="created"]',
                '*[class*="Created"]',
                '*[class*="timestamp"]',
                '*[class*="Timestamp"]'
            ]
            
            for selector in time_selectors:
                try:
                    time_elements = container.find_elements(By.CSS_SELECTOR, selector)
                    for element in time_elements:
                        time_text = element.text.strip()
                        if time_text and is_valid_time_format(time_text):
                            post_time = time_text
                            print(f"🕒 通过类名找到发布时间: {post_time}")
                            break
                    if post_time:
                        break
                except:
                    continue
        except Exception as e:
            print(f"类名时间提取失败: {e}")

    # 方法3: 通过属性查找（如title、aria-label等）
    if not post_time:
        try:
            all_elements = container.find_elements(By.CSS_SELECTOR, '*')
            for element in all_elements:
                # 检查各种属性中的时间信息
                attributes_to_check = ['title', 'aria-label', 'data-time', 'data-date', 'data-timestamp']
                for attr in attributes_to_check:
                    try:
                        attr_value = element.get_attribute(attr)
                        if attr_value and is_valid_time_format(attr_value):
                            post_time = attr_value
                            print(f"🕒 通过属性{attr}找到发布时间: {post_time}")
                            break
                    except:
                        continue
                if post_time:
                    break
        except Exception as e:
            print(f"属性时间提取失败: {e}")

    # 方法4: 通过XPath查找包含时间文本的元素
    if not post_time:
        try:
            # 查找包含时间关键词的元素
            time_keywords = ['前', 'ago', '發布', '发布', '上架', 'posted', 'created']
            for keyword in time_keywords:
                try:
                    time_elements = container.find_elements(By.XPATH, f".//*[contains(text(), '{keyword}')]")
                    for element in time_elements:
                        time_text = element.text.strip()
                        if time_text and is_valid_time_format(time_text):
                            post_time = time_text
                            print(f"🕒 通过XPath找到发布时间: {post_time}")
                            break
                    if post_time:
                        break
                except:
                    continue
        except Exception as e:
            print(f"XPath时间提取失败: {e}")

    # 方法5: 从容器文本中正则匹配
    if not post_time:
        time_patterns = [
            r'(\d+\s*(秒|秒钟|秒鐘|分鐘|分钟|mins?|小時|小时|hrs?|天|days?|週|周|weeks?|月|months?|年|years?)\s*前)',
            r'(\d+\s*(second|minute|hour|day|week|month|year)s?\s*ago)',
            r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})',
            r'(\d{1,2}:\d{2}(?::\d{2})?)',
            r'((?:今天|今日|today)\s*\d{1,2}:\d{2})',
            r'((?:昨天|昨日|yesterday)\s*\d{1,2}:\d{2})',
            r'(\d{4}-\d{1,2}-\d{1,2})',
            r'(剛剛|剛才|just now)'
        ]
        
        for pattern in time_patterns:
            time_match = re.search(pattern, container_text, re.IGNORECASE)
            if time_match:
                post_time = time_match.group(1)
                print(f"🕒 通过正则匹配找到发布时间: {post_time}")
                break

    # 方法6: 向上查找父容器中的时间信息
    if not post_time:
        try:
            current_element = container
            for i in range(5):  # 向上查找5级父容器
                try:
                    parent = current_element.find_element(By.XPATH, "./..")
                    parent_text = parent.text.strip()
                    
                    # 检查父容器文本中的时间
                    for pattern in time_patterns:
                        time_match = re.search(pattern, parent_text, re.IGNORECASE)
                        if time_match:
                            post_time = time_match.group(1)
                            print(f"🕒 通过父容器找到发布时间: {post_time}")
                            break
                    
                    if post_time:
                        break
                    
                    # 检查父容器中的时间元素
                    time_elements = parent.find_elements(By.CSS_SELECTOR, 
                        '[data-testid*="time"], [class*="time"], [class*="date"]'
                    )
                    for element in time_elements:
                        time_text = element.text.strip()
                        if time_text and is_valid_time_format(time_text):
                            post_time = time_text
                            print(f"🕒 通过父容器元素找到发布时间: {post_time}")
                            break
                    
                    if post_time:
                        break
                    
                    current_element = parent
                except:
                    break
        except Exception as e:
            print(f"父容器时间提取失败: {e}")

    return post_time

def is_valid_time_format(text):
    """验证是否为有效的时间格式"""
    time_indicators = [
        '前', 'ago', '秒', '分钟', '分鐘', '小时', '小時', '天', '日', '周', '週', '月', '年',
        'today', 'yesterday', 'just now', '剛剛', '發布', '发布', '上架'
    ]
    
    # 检查是否包含时间指示词
    has_time_indicator = any(indicator in text.lower() for indicator in time_indicators)
    
    # 检查时间格式模式
    time_patterns = [
        r'\d+\s*(秒|秒钟|秒鐘|分鐘|分钟|mins?|小時|小时|hrs?|天|days?|週|周|weeks?|月|months?|年|years?)\s*前',
        r'\d+\s*(second|minute|hour|day|week|month|year)s?\s*ago',
        r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',
        r'\d{1,2}:\d{2}',
        r'(今天|今日|today)\s*\d{1,2}:\d{2}',
        r'(昨天|昨日|yesterday)\s*\d{1,2}:\d{2}',
        r'\d{4}-\d{1,2}-\d{1,2}',
        r'剛剛|剛才|just now'
    ]
    
    has_time_pattern = any(re.search(pattern, text, re.IGNORECASE) for pattern in time_patterns)
    
    # 排除明显不是时间的内容
    not_time_indicators = ['labubu', '盲盒', '公仔', '手办', 'HK$', '$', 'seller', '賣家']
    is_not_time = any(indicator in text.lower() for indicator in not_time_indicators)
    
    return (has_time_indicator or has_time_pattern) and not is_not_time and len(text) <= 30

def extract_link(container):
    """提取商品链接"""
    link = ""
    try:
        if container.tag_name == "a":
            link = container.get_attribute("href")
        else:
            link_elements = container.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
            for element in link_elements:
                href = element.get_attribute("href")
                if href and "/p/" in href:
                    link = href
                    break
    except:
        pass
    return link

# 其余函数保持不变（extract_products_by_structure, analyze_element_structure, create_product_id, click_show_more_button, save_data_to_desktop, display_data_statistics, check_desktop_file）

def extract_products_by_structure(driver, seen_products):
    """通过页面结构分析提取商品信息"""
    products = []
    try:
        product_cards = driver.find_elements(By.CSS_SELECTOR, 'div[class*="listing-item"], div[class*="product-card"], a[href*="/p/"]')
        print(f"结构分析找到 {len(product_cards)} 个潜在商品卡片")

        for card in product_cards:
            try:
                product_info = analyze_element_structure(card)
                if product_info and product_info['title'] and len(product_info['title']) > 3:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ 结构分析提取: {product_info['title'][:30]}... | 价格: {product_info['price']} | 卖家: {product_info['seller']} | 时间: {product_info['post_time']}")
            except:
                continue

    except Exception as e:
        print(f"结构分析提取时出错: {e}")

    return products

def analyze_element_structure(element):
    """分析元素结构来提取商品信息"""
    try:
        container_text = element.text.strip()
        
        # 提取标题
        title = ""
        try:
            title_elements = element.find_elements(By.CSS_SELECTOR, '[data-testid*="title"], [class*="title"], h1, h2, h3, h4, h5, h6')
            for title_element in title_elements:
                text = title_element.text.strip()
                if text and len(text) >= 3 and len(text) <= 100:
                    title = text
                    break
        except:
            pass
        
        if not title:
            lines = container_text.split('\n')
            for line in lines:
                line = line.strip()
                if (len(line) >= 3 and len(line) <= 100
                    and not re.search(r'(HK\$|\$|\d+)', line)
                    and not re.search(r'\d+\s*(小時|小时|天|分鐘|分钟|月|年)前', line)
                    and ('labubu' in line.lower() or any(keyword in line.lower() for keyword in ['盲盒', '公仔', '手办']))
                ):
                    title = line
                    break

        if not title or len(title) < 3:
            return None

        # 提取价格
        price = ""
        price_match = re.search(r'(HK\$|\$)(\d+(?:,\d+)*(?:\.\d+)?)', container_text)
        if price_match:
            price = f"{price_match.group(1)}{price_match.group(2)}"

        # 提取卖家
        seller = ""
        try:
            seller_elements = element.find_elements(By.CSS_SELECTOR, '[data-testid*="seller"], [class*="seller"]')
            for seller_element in seller_elements:
                seller_text = seller_element.text.strip()
                if seller_text:
                    seller = seller_text
                    break
        except:
            pass

        if not seller:
            lines = container_text.split('\n')
            for line in lines:
                line = line.strip()
                if (line and line != title
                    and len(line) >= 2 and len(line) <= 20
                    and not re.search(r'(HK\$|\$|\d+)', line)
                    and not re.search(r'\d+\s*(小時|小时|天|分鐘|分钟|月|年)前', line)
                ):
                    seller = line
                    break

        # 提取发布时间
        post_time = extract_post_time_comprehensive(element, container_text)

        # 获取链接
        link = ""
        try:
            if element.tag_name == "a":
                link = element.get_attribute("href")
            else:
                link_elements = element.find_elements(By.TAG_NAME, "a")
                for link_element in link_elements:
                    href = link_element.get_attribute("href")
                    if href and "/p/" in href:
                        link = href
                        break
        except:
            pass

        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        return product_info

    except Exception as e:
        return None

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info['link']}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def click_show_more_button(driver, wait):
    """改进的翻页按钮点击"""
    try:
        show_more_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), '顯示更多結果') or contains(text(), 'Show more results')]"
        )
        for button in show_more_buttons:
            try:
                if button.is_displayed() and button.is_enabled():
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                    time.sleep(2)
                    driver.execute_script("arguments[0].click();", button)
                    print("成功点击翻页按钮")
                    return True
            except:
                continue

        try:
            driver.execute_script("""
                var buttons = document.querySelectorAll('button');
                for (var i = 0; i < buttons.length; i++) {
                    if (buttons[i].textContent.includes('顯示更多結果') ||
                        buttons[i].textContent.includes('Show more results')) {
                        buttons[i].click();
                        return true;
                    }
                }
                return false;
            """)
            print("通过JavaScript点击翻页按钮")
            return True
        except:
            pass

        print("未找到可点击的翻页按钮")
        return False

    except Exception as e:
        print(f"点击翻页按钮时出错: {e}")
        return False

def save_data_to_desktop(products):
    """保存数据到桌面，使用固定文件名"""
    if not products:
        print("没有数据可保存")
        return None

    print(f"准备保存 {len(products)} 个商品数据到桌面")
    try:
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = "carousell_labubu_products.csv"
        filepath = os.path.join(desktop_path, filename)

        df = pd.DataFrame(products)

        column_order = ['title', 'price', 'seller', 'post_time', 'link', 'crawl_time']
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]

        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户',
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        df = df.rename(columns=column_mapping)

        if os.path.exists(filepath):
            existing_df = pd.read_csv(filepath, encoding='utf-8-sig')
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df = combined_df.drop_duplicates(subset=['商品名称', '价格', '商品链接'], keep='last')
            combined_df = combined_df.sort_values('爬取时间', ascending=False)
            print(f"合并数据: 原有 {len(existing_df)} 条，新增 {len(df)} 条，去重后 {len(combined_df)} 条")
        else:
            combined_df = df
            print("创建新文件")

        combined_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 数据已保存到桌面: {filepath}")
        print(f"📊 总数据量: {len(combined_df)} 条记录")

        display_data_statistics(combined_df)
        return filepath

    except Exception as e:
        print(f"保存数据到桌面失败: {e}")
        return None

def display_data_statistics(df):
    """显示数据完整性统计"""
    print("\n" + "="*50)
    print("📈 数据完整性统计:")
    print("="*50)
    print(f"总记录数: {len(df)}")
    print(f"有商品名称: {df['商品名称'].notna().sum()}")
    print(f"有价格信息: {df['价格'].notna().sum()}")
    print(f"有发布用户: {df['发布用户'].notna().sum()}")
    print(f"有发布时间: {df['发布时间'].notna().sum()}")
    print(f"有商品链接: {df['商品链接'].notna().sum()}")

def check_desktop_file():
    """检查桌面数据文件"""
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    filepath = os.path.join(desktop_path, "carousell_labubu_products.csv")
    if os.path.exists(filepath):
        df = pd.read_csv(filepath, encoding='utf-8-sig')
        print(f"📁 桌面文件: {filepath}")
        print(f"📊 现有数据量: {len(df)} 条记录")
        print("\n📋 前5条数据详情:")
        for i, row in df.head().iterrows():
            print(f"{i+1}. 商品: {row['商品名称'][:40]}...")
            print(f"   价格: {row['价格']}")
            print(f"   卖家: {row['发布用户']}")
            print(f"   时间: {row['发布时间']}")
            print()
        return True
    else:
        print("❌ 桌面未找到数据文件")
        return False

if __name__ == "__main__":
    print("🚀 开始爬取Carousell Labubu商品...")
    print("\n" + "="*60)
    print("🔍 检查桌面数据文件:")
    print("="*60)
    check_desktop_file()

    products, saved_file = scrape_carousell_labubu()
    if products and saved_file:
        print(f"\n✅ 爬取完成！成功获取 {len(products)} 个新商品")
        print(f"💾 所有数据已保存到: {saved_file}")
    elif products:
        print(f"\n⚠️ 爬取到 {len(products)} 个商品，但文件保存失败")
    else:
        print("❌ 爬取失败")

    print("\n" + "="*60)
    print("📋 最终文件检查:")
    print("="*60)
    check_desktop_file()

🚀 开始爬取Carousell Labubu商品...

🔍 检查桌面数据文件:
❌ 桌面未找到数据文件
正在访问搜索页面: https://www.carousell.com.hk/search/labubu?addRecent=true&canChangeKeyword=true&includeSuggestions=true&srsltid=AfmBOooIk7T6Y9wa_JLsXghacyb2Qkc78nV-IKUYO8PQMMrE0ng7m8Zk&t-search_query_source=direct_search
页面body加载完成

=== 正在处理第 1 页 ===
开始滚动页面...
滚动 1/5 次
滚动 2/5 次
页面高度未变化，停止滚动
🔍 找到 0 个商品容器
📊 有效商品: 0/0
结构分析找到 47 个潜在商品卡片
🕒 通过父容器找到发布时间: 13 分鐘前
✅ 结构分析提取: [POP MART] The Monsters Labubu... | 价格: HK$630 | 卖家: 買家保障 | 时间: 13 分鐘前
🕒 通过父容器找到发布时间: 23 分鐘前
✅ 结构分析提取: Crybaby Sweet Baby Figure Set ... | 价格: HK$50 | 卖家: 幾乎全新 | 时间: 23 分鐘前
🕒 通过父容器找到发布时间: 53 分鐘前
✅ 结构分析提取: Labubu心底密碼系列字母A，換C或售... | 价格: HK$100 | 卖家: 買家保障 | 时间: 53 分鐘前
🕒 通过父容器找到发布时间: 36 分鐘前
✅ 结构分析提取: Labubu 綠植 手辦🌳... | 价格: HK$30 | 卖家: 幾乎全新 | 时间: 36 分鐘前
🕒 通过父容器找到发布时间: 41 分鐘前
✅ 结构分析提取: 小米蘭 labubu... | 价格: HK$190 | 卖家: 買家保障 | 时间: 41 分鐘前
🕒 通过父容器找到发布时间: 1 小時前
✅ 结构分析提取: Labubu Popmart 你最珍貴系列手辦 SP 密語信... | 价格: HK$95 | 卖家: 買家保障 | 时间: 1 小時前
🕒 通过父容器找到发布时间: 2 小時前
✅ 结构分析提取: Why so serious? Labub

In [ ]:
#v5 （final）修复数据爬取问题版本

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import os
from datetime import datetime
import re
import hashlib

def scrape_carousell_labubu():
    chrome_options = Options()
    # 调试阶段不使用无头模式
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    # 增加更多反检测选项
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("useAutomationExtension", False)
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    })

    all_products = []
    max_pages = 150  # 减少页数进行测试
    seen_products = set()

    try:
        # 直接访问搜索页面
        search_url = "https://www.carousell.com.hk/search/molly"
        print(f"正在访问搜索页面: {search_url}")
        driver.get(search_url)

        # 使用更长的显式等待
        wait = WebDriverWait(driver, 20)

        # 等待页面加载 - 使用多个条件
        print("等待页面加载...")
        try:
            # 等待搜索框或商品卡片加载
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder*="搜索"], [data-testid*="listing-card"], [class*="listing-card"]')))
            print("页面主要元素加载完成")
        except TimeoutException:
            print("页面加载超时，尝试继续执行")

        # 等待更长时间让内容加载
        time.sleep(5)

        current_page = 1
        while current_page <= max_pages:
            print(f"\n=== 正在处理第 {current_page} 页 ===")

            # 滚动页面以加载所有内容
            scroll_page(driver)

            # 等待新内容加载
            time.sleep(3)

            # 尝试多种提取方法
            products = []
            extract_methods = [
                extract_products_by_testid,
                extract_products_by_class,
                extract_products_by_href
            ]
            
            for method in extract_methods:
                if not products:  # 如果前面的方法没找到，尝试下一种
                    print(f"尝试方法: {method.__name__}")
                    products = method(driver, seen_products)
                    if products:
                        print(f"✅ 方法 {method.__name__} 找到 {len(products)} 个商品")
                        break
            
            if products:
                all_products.extend(products)
                print(f"第 {current_page} 页找到 {len(products)} 个新商品")
                print(f"累计找到 {len(all_products)} 个唯一商品")
            else:
                print(f"第 {current_page} 页未找到新商品")
                # 打印页面状态用于调试
                print_page_status(driver)

            # 检查是否还有更多内容
            if not click_show_more_button(driver, wait):
                print("没有找到'顯示更多結果'按钮，可能已到最后一页")
                break

            current_page += 1
            time.sleep(3)

        print(f"\n爬取完成！总共找到 {len(all_products)} 个唯一商品")

        # 保存数据到桌面
        if all_products:
            saved_file = save_data_to_desktop(all_products)
            return all_products, saved_file
        else:
            print("没有找到任何商品数据")
            return None, None

    except Exception as e:
        print(f"爬取过程中出错: {e}")
        import traceback
        traceback.print_exc()
        
        # 即使出错也尝试保存已收集的数据
        if all_products:
            saved_file = save_data_to_desktop(all_products)
            return all_products, saved_file
        return None, None
    finally:
        print("关闭浏览器...")
        driver.quit()

def print_page_status(driver):
    """打印页面状态用于调试"""
    try:
        page_title = driver.title
        page_url = driver.current_url
        page_source_length = len(driver.page_source)
        
        print(f"页面标题: {page_title}")
        print(f"页面URL: {page_url}")
        print(f"页面源码长度: {page_source_length}")
        
        # 检查是否有错误信息
        error_elements = driver.find_elements(By.CSS_SELECTOR, '[class*="error"], [class*="empty"], [class*="no-result"]')
        if error_elements:
            print("检测到可能的错误元素:")
            for elem in error_elements[:3]:  # 只显示前3个
                print(f"  - {elem.text[:100]}")
                
        # 检查是否有任何商品相关元素
        product_like_elements = driver.find_elements(By.CSS_SELECTOR, 'div, a, article, section')
        product_count = 0
        for elem in product_like_elements[:50]:  # 只检查前50个元素
            text = elem.text.strip()
            if text and ('molly' in text.lower() or 'Molly' in text):
                product_count += 1
                print(f"找到包含molly的元素: {text[:50]}...")
        
        print(f"找到 {product_count} 个包含molly文本的元素")
        
    except Exception as e:
        print(f"检查页面状态时出错: {e}")

def scroll_page(driver):
    """改进的页面滚动"""
    print("开始滚动页面...")
    
    # 先滚动到页面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    # 再滚动回顶部然后慢慢向下滚动
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)
    
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = scroll_height // 5
    
    for i in range(5):
        scroll_to = scroll_step * (i + 1)
        driver.execute_script(f"window.scrollTo(0, {scroll_to});")
        print(f"滚动 {i+1}/5 次，位置: {scroll_to}")
        time.sleep(2)

def extract_products_by_testid(driver, seen_products):
    """通过data-testid属性提取商品"""
    products = []
    try:
        # 多种可能的商品容器选择器
        selectors = [
            '[data-testid="listing-card"]',
            '[data-testid*="listing"]',
            '[data-testid*="product"]',
            '[data-testid*="item"]'
        ]
        
        all_containers = []
        for selector in selectors:
            containers = driver.find_elements(By.CSS_SELECTOR, selector)
            all_containers.extend(containers)
            if containers:
                print(f"使用选择器 '{selector}' 找到 {len(containers)} 个容器")
        
        # 去重
        unique_containers = []
        seen_texts = set()
        for container in all_containers:
            try:
                text = container.text.strip()
                if text and text not in seen_texts:
                    seen_texts.add(text)
                    unique_containers.append(container)
            except:
                continue
        
        print(f"去重后找到 {len(unique_containers)} 个唯一容器")
        
        for i, container in enumerate(unique_containers):
            try:
                product_info = extract_product_info(container)
                if product_info and product_info['title'] and len(product_info['title']) > 3:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ [{i+1}/{len(unique_containers)}] 提取: {product_info['title'][:30]}... | 价格: {product_info['price']}")
            except Exception as e:
                print(f"❌ [{i+1}/{len(unique_containers)}] 提取失败: {e}")
                continue
                
    except Exception as e:
        print(f"通过data-testid提取商品时出错: {e}")
        
    return products

def extract_products_by_class(driver, seen_products):
    """通过类名提取商品"""
    products = []
    try:
        # 多种可能的类名选择器
        selectors = [
            'div[class*="listingCard"]',
            'div[class*="productCard"]',
            'div[class*="listing"]',
            'div[class*="card"]',
            'a[class*="listing"]',
            'a[class*="product"]',
            'article[class*="listing"]'
        ]
        
        all_containers = []
        for selector in selectors:
            containers = driver.find_elements(By.CSS_SELECTOR, selector)
            all_containers.extend(containers)
            if containers:
                print(f"使用类名选择器 '{selector}' 找到 {len(containers)} 个容器")
        
        # 去重
        unique_containers = []
        seen_texts = set()
        for container in all_containers:
            try:
                text = container.text.strip()
                if text and text not in seen_texts and len(text) > 20:  # 过滤掉太短的文本
                    seen_texts.add(text)
                    unique_containers.append(container)
            except:
                continue
        
        print(f"类名选择器去重后找到 {len(unique_containers)} 个唯一容器")
        
        for i, container in enumerate(unique_containers):
            try:
                product_info = extract_product_info(container)
                if product_info and product_info['title'] and len(product_info['title']) > 3:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ [{i+1}/{len(unique_containers)}] 类名提取: {product_info['title'][:30]}...")
            except Exception as e:
                continue
                
    except Exception as e:
        print(f"通过类名提取商品时出错: {e}")
        
    return products

def extract_products_by_href(driver, seen_products):
    """通过链接模式提取商品"""
    products = []
    try:
        # 查找包含商品链接的元素
        link_elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
        print(f"找到 {len(link_elements)} 个商品链接")
        
        for i, link_element in enumerate(link_elements):
            try:
                # 获取链接元素的父容器（可能是商品卡片）
                container = link_element
                for _ in range(3):  # 向上查找3级父元素
                    try:
                        container = container.find_element(By.XPATH, "./..")
                        if container.text and len(container.text.strip()) > 20:
                            break
                    except:
                        break
                
                product_info = extract_product_info(container)
                if product_info and product_info['title'] and len(product_info['title']) > 3:
                    product_id = create_product_id(product_info)
                    if product_id not in seen_products:
                        seen_products.add(product_id)
                        products.append(product_info)
                        print(f"✅ [{i+1}/{len(link_elements)}] 链接提取: {product_info['title'][:30]}...")
            except:
                continue
                
    except Exception as e:
        print(f"通过链接提取商品时出错: {e}")
        
    return products

def extract_product_info(container):
    """从容器提取商品信息"""
    try:
        container_text = container.text.strip()
        if not container_text or len(container_text) < 20:
            return None
        
        # 按行分割文本
        lines = [line.strip() for line in container_text.split('\n') if line.strip()]
        
        if len(lines) < 3:
            return None
        
        # 分析文本结构提取信息
        seller = ""
        post_time = ""
        title = ""
        price = ""
        
        # 发布用户：通常是第一行
        if len(lines) > 0:
            seller = lines[0]
        
        # 发布时间：查找包含时间格式的行
        for i, line in enumerate(lines):
            if is_valid_time_format(line):
                post_time = line
                # 如果发布时间不在第二行，可能需要调整用户位置
                if i != 1 and i > 0:
                    # 可能是第一行是其他信息，第二行是用户
                    seller = lines[i-1] if i-1 >= 0 else seller
                break
        
        # 商品标题：查找包含molly关键词的行
        for line in lines:
            if ('molly' in line.lower() or 
                any(keyword in line.lower() for keyword in ['盲盒', '公仔', '手办'])):
                title = line
                break
        
        # 如果没有找到标题，使用最长的行作为备选
        if not title and lines:
            title = max(lines, key=len)
        
        # 价格：查找包含价格符号的行
        for line in lines:
            if re.search(r'(HK\$|\$|FREE|免費|免费)', line, re.IGNORECASE):
                price_match = re.search(r'(HK\$|\$|FREE|免費|免费)\s*(\d+(?:,\d+)*(?:\.\d+)?)?', line, re.IGNORECASE)
                if price_match:
                    if price_match.group(1).upper() in ['FREE', '免費', '免费']:
                        price = "FREE"
                    else:
                        price = f"{price_match.group(1)}{price_match.group(2) if price_match.group(2) else ''}"
                break
        
        # 获取商品链接
        link = ""
        try:
            if container.tag_name == "a":
                link = container.get_attribute("href")
            else:
                link_elements = container.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
                for link_element in link_elements:
                    href = link_element.get_attribute("href")
                    if href and "/p/" in href:
                        link = href
                        break
        except:
            pass

        # 验证必要信息
        if not title or len(title) < 3:
            return None

        product_info = {
            'title': title,
            'price': price,
            'seller': seller,
            'post_time': post_time,
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        return product_info

    except Exception as e:
        print(f"提取商品信息时出错: {e}")
        return None

def is_valid_time_format(text):
    """验证是否为有效的时间格式"""
    if not text:
        return False
        
    time_patterns = [
        r'\d+\s*(秒|秒钟|秒鐘|分鐘|分钟|mins?|小時|小时|hrs?|天|days?|週|周|weeks?|月|months?|年|years?)\s*前',
        r'\d+\s*(second|minute|hour|day|week|month|year)s?\s*ago',
        r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',
        r'\d{1,2}:\d{2}',
        r'(今天|今日|today)\s*\d{1,2}:\d{2}',
        r'(昨天|昨日|yesterday)\s*\d{1,2}:\d{2}',
        r'剛剛|剛才|just now'
    ]
    
    has_time_pattern = any(re.search(pattern, text, re.IGNORECASE) for pattern in time_patterns)
    
    # 排除明显不是时间的内容
    not_time_indicators = ['labubu', '盲盒', '公仔', '手办', 'HK$', '$']
    is_not_time = any(indicator in text.lower() for indicator in not_time_indicators)
    
    return has_time_pattern and not is_not_time and len(text) <= 30

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info['link']}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def click_show_more_button(driver, wait):
    """改进的翻页按钮点击"""
    try:
        # 方法1: 通过按钮文本
        show_more_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), '顯示更多結果') or contains(text(), 'Show more results') or contains(text(), 'Load more')]"
        )
        
        for button in show_more_buttons:
            try:
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                time.sleep(2)
                
                if button.is_displayed() and button.is_enabled():
                    driver.execute_script("arguments[0].click();", button)
                    print("成功点击翻页按钮")
                    time.sleep(3)  # 等待新内容加载
                    return True
            except:
                continue

        # 方法2: 通过JavaScript直接查找并点击
        try:
            result = driver.execute_script("""
                var buttons = document.querySelectorAll('button');
                for (var i = 0; i < buttons.length; i++) {
                    var btn = buttons[i];
                    var text = btn.textContent || btn.innerText;
                    if (text.includes('顯示更多結果') || text.includes('Show more results') || text.includes('Load more')) {
                        btn.click();
                        return true;
                    }
                }
                return false;
            """)
            if result:
                print("通过JavaScript点击翻页按钮")
                time.sleep(3)
                return True
        except:
            pass

        print("未找到可点击的翻页按钮")
        return False

    except Exception as e:
        print(f"点击翻页按钮时出错: {e}")
        return False

def save_data_to_desktop(products):
    """保存数据到桌面"""
    if not products:
        print("没有数据可保存")
        return None

    print(f"准备保存 {len(products)} 个商品数据到桌面")
    try:
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = "carousell_labubu_products.csv"
        filepath = os.path.join(desktop_path, filename)

        df = pd.DataFrame(products)

        column_order = ['title', 'price', 'seller', 'post_time', 'link', 'crawl_time']
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]

        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户',
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        df = df.rename(columns=column_mapping)

        if os.path.exists(filepath):
            existing_df = pd.read_csv(filepath, encoding='utf-8-sig')
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df = combined_df.drop_duplicates(subset=['商品名称', '价格', '商品链接'], keep='last')
            combined_df = combined_df.sort_values('爬取时间', ascending=False)
            print(f"合并数据: 原有 {len(existing_df)} 条，新增 {len(df)} 条，去重后 {len(combined_df)} 条")
        else:
            combined_df = df
            print("创建新文件")

        combined_df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 数据已保存到桌面: {filepath}")
        print(f"📊 总数据量: {len(combined_df)} 条记录")

        display_data_statistics(combined_df)
        return filepath

    except Exception as e:
        print(f"保存数据到桌面失败: {e}")
        return None

def display_data_statistics(df):
    """显示数据完整性统计"""
    print("\n" + "="*50)
    print("📈 数据完整性统计:")
    print("="*50)
    print(f"总记录数: {len(df)}")
    print(f"有商品名称: {df['商品名称'].notna().sum()}")
    print(f"有价格信息: {df['价格'].notna().sum()}")
    print(f"有发布用户: {df['发布用户'].notna().sum()}")
    print(f"有发布时间: {df['发布时间'].notna().sum()}")
    print(f"有商品链接: {df['商品链接'].notna().sum()}")

def check_desktop_file():
    """检查桌面数据文件"""
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    filepath = os.path.join(desktop_path, "carousell_labubu_products.csv")
    if os.path.exists(filepath):
        df = pd.read_csv(filepath, encoding='utf-8-sig')
        print(f"📁 桌面文件: {filepath}")
        print(f"📊 现有数据量: {len(df)} 条记录")
        if len(df) > 0:
            print("\n📋 前5条数据详情:")
            for i, row in df.head().iterrows():
                print(f"{i+1}. 商品: {row['商品名称'][:40]}...")
                print(f"   价格: {row['价格']}")
                print(f"   卖家: {row['发布用户']}")
                print(f"   时间: {row['发布时间']}")
                print()
        return True
    else:
        print("❌ 桌面未找到数据文件")
        return False

if __name__ == "__main__":
    print("🚀 开始爬取Carousell molly商品...")
    print("\n" + "="*60)
    print("🔍 检查桌面数据文件:")
    print("="*60)
    check_desktop_file()

    products, saved_file = scrape_carousell_labubu()
    if products and saved_file:
        print(f"\n✅ 爬取完成！成功获取 {len(products)} 个新商品")
        print(f"💾 所有数据已保存到: {saved_file}")
    elif products:
        print(f"\n⚠️ 爬取到 {len(products)} 个商品，但文件保存失败")
    else:
        print("❌ 爬取失败")

    print("\n" + "="*60)
    print("📋 最终文件检查:")
    print("="*60)
    check_desktop_file()

🚀 开始爬取Carousell molly商品...

🔍 检查桌面数据文件:
❌ 桌面未找到数据文件
正在访问搜索页面: https://www.carousell.com.hk/search/molly
等待页面加载...
页面主要元素加载完成

=== 正在处理第 1 页 ===
开始滚动页面...
滚动 1/5 次，位置: 1272
滚动 2/5 次，位置: 2544
滚动 3/5 次，位置: 3816
滚动 4/5 次，位置: 5088
滚动 5/5 次，位置: 6360
尝试方法: extract_products_by_testid
使用选择器 '[data-testid*="listing"]' 找到 141 个容器
去重后找到 1 个唯一容器
尝试方法: extract_products_by_class
使用类名选择器 'div[class*="listing"]' 找到 1 个容器
类名选择器去重后找到 0 个唯一容器
尝试方法: extract_products_by_href
找到 47 个商品链接
第 1 页未找到新商品
页面标题: molly-查看在旋轉拍賣Hong Kong市場的molly個廣告
页面URL: https://www.carousell.com.hk/search/molly/
页面源码长度: 511928
找到 0 个包含molly文本的元素
成功点击翻页按钮

=== 正在处理第 2 页 ===
开始滚动页面...
滚动 1/5 次，位置: 2406
滚动 2/5 次，位置: 4812
滚动 3/5 次，位置: 7218
滚动 4/5 次，位置: 9624
滚动 5/5 次，位置: 12030
尝试方法: extract_products_by_testid
使用选择器 '[data-testid*="listing"]' 找到 288 个容器
去重后找到 200 个唯一容器
✅ [1/200] 提取: 600+全好評🔥 最齊現貨Crybaby 買2送1（送既禮品... | 价格: HK$159
✅ [4/200] 提取: 600+全好評🔥現貨Molly factory 描述有價錢👇... | 价格: HK$460
✅ [6/200] 提取: Apple of my eye (Molly)... | 价格: HK$

Traceback (most recent call last):
  File "c:\Users\86155\anaconda3\envs\hazel\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\86155\anaconda3\envs\hazel\Lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\86155\anaconda3\envs\hazel\Lib\http\client.py", line 1395, in getresponse
    response.begin()
  File "c:\Users\86155\anaconda3\envs\hazel\Lib\http\client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\86155\anaconda3\envs\hazel\Lib\http\client.py", line 286, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\86155\anaconda3\envs\hazel\Lib\socket.py", line 718, in readinto
   


✅ 爬取完成！成功获取 575 个新商品
💾 所有数据已保存到: C:\Users\86155\Desktop\carousell_labubu_products.csv

📋 最终文件检查:
📁 桌面文件: C:\Users\86155\Desktop\carousell_labubu_products.csv
📊 现有数据量: 575 条记录

📋 前5条数据详情:
1. 商品: 600+全好評🔥 最齊現貨Crybaby 買2送1（送既禮品一份 請睇描述）官方...
   价格: HK$159
   卖家: lovesouryellowlemon
   时间: 12 分鐘前

2. 商品: 600+全好評🔥現貨Molly factory 描述有價錢👇有優惠 買兩款起有優...
   价格: HK$460
   卖家: lovesouryellowlemon
   时间: 13 分鐘前

3. 商品: Apple of my eye (Molly)...
   价格: HK$60
   卖家: 911011
   时间: 1 小時前

4. 商品: Molly Apple of My Eye Series Candlelight...
   价格: HK$90
   卖家: verasin1289589
   时间: 1 小時前

5. 商品: POP MART MEGA SPACE MOLLY 宇宙系列...
   价格: HK$130
   卖家: edccm
   时间: 1 小時前



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import os
from datetime import datetime
import re
import hashlib
import random
import requests
from bs4 import BeautifulSoup
import json

def setup_driver_advanced():
    """高级浏览器设置 - 绕过检测"""
    chrome_options = Options()
    
    # 基础设置
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    
    # 高级反检测设置
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # 禁用图片和CSS加速加载
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-plugins")
    chrome_options.add_argument("--disable-javascript")
    
    # 随机用户代理
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    ]
    selected_ua = random.choice(user_agents)
    chrome_options.add_argument(f"--user-agent={selected_ua}")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    # 执行高级反检测脚本
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3, 4, 5]})")
    driver.execute_script("Object.defineProperty(navigator, 'languages', {get: () => ['zh-CN', 'zh', 'en']})")
    
    # 设置超时
    driver.set_page_load_timeout(30)
    driver.set_script_timeout(30)
    
    return driver

def scrape_carousell_advanced():
    """高级爬虫 - 突破500条限制"""
    driver = None
    all_products = []
    seen_products = set()
    
    try:
        driver = setup_driver_advanced()
        
        # 策略1: 使用不同的搜索URL和参数
        search_urls = [
            "https://www.carousell.com.hk/search/molly",
            "https://www.carousell.com.hk/search/molly?sort_by=time_created%2Cdescending",
            "https://www.carousell.com.hk/search/molly?sort_by=price%2Cascending",
            "https://www.carousell.com.hk/search/molly?addRecent=false"
        ]
        
        for search_url in search_urls:
            if len(all_products) >= 1000:  # 如果已经收集足够多数据
                break
                
            print(f"\n🎯 尝试URL: {search_url}")
            products_from_url = scrape_with_url(driver, search_url, seen_products)
            all_products.extend(products_from_url)
            print(f"从该URL获得 {len(products_from_url)} 个商品")
            
            # 休息一下避免被封
            time.sleep(random.uniform(5, 10))
        
        print(f"\n总共收集到 {len(all_products)} 个唯一商品")
        
        if all_products:
            saved_file = save_advanced_data(all_products)
            return all_products, saved_file
        else:
            return None, None

    except Exception as e:
        print(f"高级爬取过程中出错: {e}")
        import traceback
        traceback.print_exc()
        
        if all_products:
            saved_file = save_advanced_data(all_products)
            return all_products, saved_file
        return None, None
    finally:
        if driver:
            driver.quit()

def scrape_with_url(driver, search_url, seen_products):
    """使用特定URL爬取"""
    products = []
    
    try:
        print(f"访问: {search_url}")
        driver.get(search_url)
        
        # 等待页面加载
        time.sleep(5)
        
        # 模拟人类行为滚动
        human_like_scroll(driver)
        
        max_pages = 20  # 每URL最多20页
        current_page = 1
        
        while current_page <= max_pages:
            print(f"处理第 {current_page} 页...")
            
            # 深度滚动加载更多内容
            deep_scroll_for_content(driver)
            
            # 提取当前页商品
            page_products = extract_products_advanced(driver, seen_products)
            products.extend(page_products)
            
            print(f"第 {current_page} 页找到 {len(page_products)} 个新商品")
            
            # 检查是否还有更多页面
            if not has_more_content(driver):
                print("没有更多内容，停止翻页")
                break
                
            # 尝试翻页
            if not smart_pagination(driver, current_page):
                print("无法翻页，停止")
                break
                
            current_page += 1
            
            # 随机延迟，模拟人类行为
            time.sleep(random.uniform(3, 8))
            
            # 每5页休息更长时间
            if current_page % 5 == 0:
                print("休息一下...")
                time.sleep(random.uniform(10, 15))
    
    except Exception as e:
        print(f"该URL爬取失败: {e}")
    
    return products

def human_like_scroll(driver):
    """模拟人类滚动行为"""
    print("模拟人类滚动...")
    
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    viewport_height = driver.execute_script("return window.innerHeight")
    
    # 分段滚动，带有随机停顿
    scroll_positions = [
        viewport_height * 0.3,
        viewport_height * 0.6,
        viewport_height * 0.9,
        scroll_height * 0.2,
        scroll_height * 0.4,
        scroll_height * 0.6,
        scroll_height * 0.8,
        scroll_height
    ]
    
    for i, position in enumerate(scroll_positions):
        # 随机滚动速度
        scroll_script = f"""
        window.scrollTo({{
            top: {position},
            behavior: 'smooth'
        }});
        """
        driver.execute_script(scroll_script)
        
        # 随机等待时间
        wait_time = random.uniform(0.5, 2.5)
        time.sleep(wait_time)
        
        # 偶尔随机小范围滚动
        if random.random() < 0.3:
            small_scroll = random.randint(-100, 100)
            driver.execute_script(f"window.scrollBy(0, {small_scroll})")
            time.sleep(0.5)

def deep_scroll_for_content(driver):
    """深度滚动以加载更多内容"""
    print("深度滚动加载内容...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    no_change_count = 0
    
    for i in range(10):  # 最多尝试10次
        # 滚动到底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # 检查是否有"Load More"按钮并点击
        if click_load_more_buttons(driver):
            time.sleep(3)
        
        # 检查高度是否变化
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            no_change_count += 1
            if no_change_count >= 3:
                print("连续3次滚动高度无变化，停止滚动")
                break
        else:
            no_change_count = 0
            print(f"发现新内容，页面高度: {new_height}")
        
        last_height = new_height
        
        # 随机等待
        time.sleep(random.uniform(1, 3))

def click_load_more_buttons(driver):
    """点击所有可能的加载更多按钮"""
    load_more_selectors = [
        "button[data-testid*='load-more']",
        "button:contains('Load More')",
        "button:contains('顯示更多')",
        "button:contains('Show More')",
        ".load-more",
        "[class*='load-more']"
    ]
    
    for selector in load_more_selectors:
        try:
            # 使用XPath包含文本的查找
            buttons = driver.find_elements(By.XPATH, 
                f"//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), '顯示更多')]")
            
            for button in buttons:
                try:
                    if button.is_displayed() and button.is_enabled():
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", button)
                        print("✅ 点击了加载更多按钮")
                        return True
                except:
                    continue
        except:
            continue
    
    return False

def extract_products_advanced(driver, seen_products):
    """高级商品提取"""
    products = []
    
    # 多种元素定位策略
    extraction_strategies = [
        extract_by_data_testid,
        extract_by_class_patterns,
        extract_by_link_analysis,
        extract_by_structural_analysis
    ]
    
    all_found_products = []
    
    for strategy in extraction_strategies:
        try:
            found_products = strategy(driver, seen_products)
            if found_products:
                all_found_products.extend(found_products)
                print(f"策略 {strategy.__name__} 找到 {len(found_products)} 个商品")
                
                # 如果找到很多商品，可以提前停止
                if len(all_found_products) > 50:
                    break
        except Exception as e:
            print(f"策略 {strategy.__name__} 失败: {e}")
            continue
    
    # 去重并添加到结果
    for product in all_found_products:
        product_id = create_product_id(product)
        if product_id not in seen_products:
            seen_products.add(product_id)
            products.append(product)
    
    return products

def extract_by_data_testid(driver, seen_products):
    """通过data-testid属性提取"""
    products = []
    
    testid_patterns = [
        'listing-card',
        'product-card',
        'listing',
        'product',
        'item'
    ]
    
    for pattern in testid_patterns:
        selector = f'[data-testid*="{pattern}"]'
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                product = extract_from_element(element)
                if product and product not in products:
                    products.append(product)
        except:
            continue
    
    return products

def extract_by_class_patterns(driver, seen_products):
    """通过类名模式提取"""
    products = []
    
    class_patterns = [
        'listingCard',
        'productCard',
        'listing',
        'card',
        'item'
    ]
    
    for pattern in class_patterns:
        selectors = [
            f'div[class*="{pattern}"]',
            f'article[class*="{pattern}"]',
            f'section[class*="{pattern}"]',
            f'a[class*="{pattern}"]'
        ]
        
        for selector in selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    product = extract_from_element(element)
                    if product and product not in products:
                        products.append(product)
            except:
                continue
    
    return products

def extract_by_link_analysis(driver, seen_products):
    """通过链接分析提取"""
    products = []
    
    try:
        # 查找所有商品链接
        link_elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
        
        for link_element in link_elements:
            try:
                # 获取链接周围的容器
                container = link_element
                for i in range(3):
                    try:
                        container = container.find_element(By.XPATH, "./..")
                        if container.text and len(container.text.strip()) > 30:
                            product = extract_from_element(container)
                            if product and product not in products:
                                products.append(product)
                            break
                    except:
                        break
            except:
                continue
                
    except Exception as e:
        print(f"链接分析失败: {e}")
    
    return products

def extract_by_structural_analysis(driver, seen_products):
    """通过结构分析提取"""
    products = []
    
    try:
        # 查找包含商品信息的结构块
        structural_selectors = [
            'div > div > div',  # 三层div结构
            'article > div',
            'section > div',
            '[role="listitem"]'
        ]
        
        for selector in structural_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    text = element.text.strip()
                    # 检查是否包含商品特征
                    if (len(text) > 50 and 
                        ('molly' in text.lower() or 'HK$' in text or '$' in text)):
                        product = extract_from_element(element)
                        if product and product not in products:
                            products.append(product)
            except:
                continue
                
    except Exception as e:
        print(f"结构分析失败: {e}")
    
    return products

def extract_from_element(element):
    """从元素提取商品信息"""
    try:
        text = element.text.strip()
        if len(text) < 30:
            return None
        
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        if len(lines) < 3:
            return None
        
        # 提取信息
        title = find_title(lines)
        price = find_price(lines)
        seller = find_seller(lines)
        time_text = find_time(lines)
        link = find_link(element)
        
        if not title:
            return None
            
        return {
            'title': clean_text(title),
            'price': price or '價格待詢',
            'seller': clean_text(seller) if seller else '',
            'post_time': clean_text(time_text) if time_text else '',
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
    except Exception as e:
        return None

def find_title(lines):
    """查找标题"""
    # 优先查找包含molly的行
    for line in lines:
        if 'molly' in line.lower() and 10 < len(line) < 100:
            return line
    
    # 返回最长的合适行
    candidate_lines = [line for line in lines if 20 < len(line) < 150]
    if candidate_lines:
        return max(candidate_lines, key=len)
    
    return lines[0] if lines else ""

def find_price(lines):
    """查找价格"""
    for line in lines:
        price_match = re.search(r'(HK\$|\$)\s*(\d+(?:,\d+)*(?:\.\d+)?)', line)
        if price_match:
            return f"{price_match.group(1)}{price_match.group(2)}"
    
    for line in lines:
        if 'free' in line.lower() or '免費' in line:
            return "FREE"
    
    return ""

def find_seller(lines):
    """查找卖家"""
    if len(lines) > 0:
        return lines[0]
    return ""

def find_time(lines):
    """查找时间"""
    time_patterns = [
        r'\d+\s*(分鐘|分钟|小時|小时|天|日|月|年)\s*前',
        r'\d+\s*(minute|hour|day|week|month|year)s?\s*ago',
        r'剛剛|剛才|just now'
    ]
    
    for line in lines:
        for pattern in time_patterns:
            if re.search(pattern, line, re.IGNORECASE):
                return line
    return ""

def find_link(element):
    """查找链接"""
    try:
        if element.tag_name == "a":
            return element.get_attribute("href")
        else:
            link_elem = element.find_element(By.CSS_SELECTOR, 'a[href*="/p/"]')
            return link_elem.get_attribute("href") if link_elem else ""
    except:
        return ""

def clean_text(text):
    """清理文本"""
    return re.sub(r'\s+', ' ', text).strip()

def has_more_content(driver):
    """检查是否还有更多内容"""
    try:
        # 检查加载更多按钮
        load_more_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), '顯示更多')]"
        )
        if load_more_buttons:
            return True
        
        # 检查下一页按钮
        next_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Next') or contains(text(), '下一頁')]"
        )
        if next_buttons:
            return True
            
        # 检查是否到达底部
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        client_height = driver.execute_script("return document.documentElement.clientHeight")
        scroll_top = driver.execute_script("return document.documentElement.scrollTop")
        
        if scroll_top + client_height < scroll_height - 100:
            return True
            
    except:
        pass
    
    return False

def smart_pagination(driver, current_page):
    """智能翻页"""
    try:
        # 方法1: 点击下一页按钮
        next_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Next') or contains(text(), '下一頁')]"
        )
        
        for button in next_buttons:
            try:
                if button.is_displayed() and button.is_enabled():
                    driver.execute_script("arguments[0].click();", button)
                    print("✅ 点击下一页按钮")
                    time.sleep(3)
                    return True
            except:
                continue
        
        # 方法2: URL翻页
        next_url = f"https://www.carousell.com.hk/search/molly?page={current_page + 1}"
        try:
            driver.get(next_url)
            print(f"✅ 跳转到第 {current_page + 1} 页")
            time.sleep(3)
            return True
        except:
            pass
            
        # 方法3: 滚动加载
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"翻页失败: {e}")
        return False

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info.get('link', '')}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def save_advanced_data(products):
    """保存高级数据"""
    if not products:
        return None
        
    try:
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = f"carousell_molly_advanced_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        filepath = os.path.join(desktop_path, filename)
        
        df = pd.DataFrame(products)
        
        # 列映射
        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户', 
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        
        df = df.rename(columns=column_mapping)
        available_columns = [col for col in column_mapping.values() if col in df.columns]
        df = df[available_columns]
        
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 高级数据已保存: {filepath}")
        print(f"📊 总数据量: {len(df)} 条记录")
        
        return filepath
        
    except Exception as e:
        print(f"保存失败: {e}")
        return None

def run_mass_scraper():
    """运行大规模爬取"""
    print("🚀 开始大规模爬取Carousell Molly商品...")
    print("🎯 目标: 突破500条限制，获取1000+条数据")
    
    all_products = []
    max_attempts = 3
    
    for attempt in range(max_attempts):
        print(f"\n{'='*60}")
        print(f"尝试 {attempt + 1}/{max_attempts}")
        print(f"{'='*60}")
        
        products, saved_file = scrape_carousell_advanced()
        
        if products:
            all_products.extend(products)
            print(f"✅ 本次获取 {len(products)} 条数据")
            print(f"📊 累计获取 {len(all_products)} 条数据")
            
            if len(all_products) >= 800:
                print("🎉 已达到目标数据量!")
                break
        else:
            print("❌ 本次爬取失败")
        
        # 长时间休息避免被封
        if attempt < max_attempts - 1:
            wait_time = random.randint(30, 60)
            print(f"⏰ 等待 {wait_time} 秒后继续...")
            time.sleep(wait_time)
    
    # 最终保存
    if all_products:
        final_file = save_advanced_data(all_products)
        print(f"\n🎉 最终结果: 成功获取 {len(all_products)} 条数据!")
        print(f"💾 保存到: {final_file}")
        return all_products, final_file
    else:
        print("❌ 所有尝试都失败了")
        return None, None

if __name__ == "__main__":
    # 运行大规模爬取
    products, saved_file = run_mass_scraper()

🚀 开始大规模爬取Carousell Molly商品...
🎯 目标: 突破500条限制，获取1000+条数据

尝试 1/3

🎯 尝试URL: https://www.carousell.com.hk/search/molly
访问: https://www.carousell.com.hk/search/molly
模拟人类滚动...
处理第 1 页...
深度滚动加载内容...
✅ 点击了加载更多按钮
发现新内容，页面高度: 13376
✅ 点击了加载更多按钮
发现新内容，页面高度: 19316
✅ 点击了加载更多按钮
发现新内容，页面高度: 25716
✅ 点击了加载更多按钮
发现新内容，页面高度: 32138
✅ 点击了加载更多按钮
发现新内容，页面高度: 38372
✅ 点击了加载更多按钮
发现新内容，页面高度: 44124
✅ 点击了加载更多按钮
发现新内容，页面高度: 50336
✅ 点击了加载更多按钮
发现新内容，页面高度: 56714
✅ 点击了加载更多按钮
发现新内容，页面高度: 62432
✅ 点击了加载更多按钮
发现新内容，页面高度: 68810
策略 extract_by_data_testid 找到 1054 个商品
第 1 页找到 527 个新商品
✅ 跳转到第 2 页
处理第 2 页...
深度滚动加载内容...
✅ 点击了加载更多按钮
发现新内容，页面高度: 13492
✅ 点击了加载更多按钮
发现新内容，页面高度: 20246
✅ 点击了加载更多按钮
发现新内容，页面高度: 26494
✅ 点击了加载更多按钮
发现新内容，页面高度: 33270
✅ 点击了加载更多按钮
发现新内容，页面高度: 39980
✅ 点击了加载更多按钮
发现新内容，页面高度: 46162
✅ 点击了加载更多按钮
发现新内容，页面高度: 52872
✅ 点击了加载更多按钮
发现新内容，页面高度: 59582
✅ 点击了加载更多按钮
发现新内容，页面高度: 66358
✅ 点击了加载更多按钮
发现新内容，页面高度: 72034
策略 extract_by_data_testid 找到 1050 个商品
第 2 页找到 525 个新商品
✅ 跳转到第 3 页
处理第 3 页...
深度滚动加载内容...
✅ 点击了加载更多按钮
发现新内容，页面高度: 134

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import os
from datetime import datetime
import re
import hashlib
import random
import requests
from bs4 import BeautifulSoup
import json

def setup_driver_advanced():
    """高级浏览器设置 - 绕过检测"""
    chrome_options = Options()
    
    # 基础设置
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    
    # 高级反检测设置
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # 禁用图片和CSS加速加载
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-plugins")
    chrome_options.add_argument("--disable-javascript")
    
    # 随机用户代理
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    ]
    selected_ua = random.choice(user_agents)
    chrome_options.add_argument(f"--user-agent={selected_ua}")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    # 执行高级反检测脚本
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3, 4, 5]})")
    driver.execute_script("Object.defineProperty(navigator, 'languages', {get: () => ['zh-CN', 'zh', 'en']})")
    
    # 设置超时
    driver.set_page_load_timeout(30)
    driver.set_script_timeout(30)
    
    return driver

def scrape_carousell_advanced():
    """高级爬虫 - 突破500条限制"""
    driver = None
    all_products = []
    seen_products = set()
    
    try:
        driver = setup_driver_advanced()
        
        # 策略1: 使用不同的搜索URL和参数
        search_urls = [
            "https://www.carousell.com.hk/search/dimoo",
            "https://www.carousell.com.hk/search/dimoo?sort_by=time_created%2Cdescending",
            "https://www.carousell.com.hk/search/dimoo?sort_by=price%2Cascending",
            "https://www.carousell.com.hk/search/dimoo?addRecent=false"
        ]
        
        for search_url in search_urls:
            if len(all_products) >= 1000:  # 如果已经收集足够多数据
                break
                
            print(f"\n🎯 尝试URL: {search_url}")
            products_from_url = scrape_with_url(driver, search_url, seen_products)
            all_products.extend(products_from_url)
            print(f"从该URL获得 {len(products_from_url)} 个商品")
            
            # 休息一下避免被封
            time.sleep(random.uniform(5, 10))
        
        print(f"\n总共收集到 {len(all_products)} 个唯一商品")
        
        if all_products:
            saved_file = save_advanced_data(all_products)
            return all_products, saved_file
        else:
            return None, None

    except Exception as e:
        print(f"高级爬取过程中出错: {e}")
        import traceback
        traceback.print_exc()
        
        if all_products:
            saved_file = save_advanced_data(all_products)
            return all_products, saved_file
        return None, None
    finally:
        if driver:
            driver.quit()

def scrape_with_url(driver, search_url, seen_products):
    """使用特定URL爬取"""
    products = []
    
    try:
        print(f"访问: {search_url}")
        driver.get(search_url)
        
        # 等待页面加载
        time.sleep(5)
        
        # 模拟人类行为滚动
        human_like_scroll(driver)
        
        max_pages = 20  # 每URL最多20页
        current_page = 1
        
        while current_page <= max_pages:
            print(f"处理第 {current_page} 页...")
            
            # 深度滚动加载更多内容
            deep_scroll_for_content(driver)
            
            # 提取当前页商品
            page_products = extract_products_advanced(driver, seen_products)
            products.extend(page_products)
            
            print(f"第 {current_page} 页找到 {len(page_products)} 个新商品")
            
            # 检查是否还有更多页面
            if not has_more_content(driver):
                print("没有更多内容，停止翻页")
                break
                
            # 尝试翻页
            if not smart_pagination(driver, current_page):
                print("无法翻页，停止")
                break
                
            current_page += 1
            
            # 随机延迟，模拟人类行为
            time.sleep(random.uniform(3, 8))
            
            # 每5页休息更长时间
            if current_page % 5 == 0:
                print("休息一下...")
                time.sleep(random.uniform(10, 15))
    
    except Exception as e:
        print(f"该URL爬取失败: {e}")
    
    return products

def human_like_scroll(driver):
    """模拟人类滚动行为"""
    print("模拟人类滚动...")
    
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    viewport_height = driver.execute_script("return window.innerHeight")
    
    # 分段滚动，带有随机停顿
    scroll_positions = [
        viewport_height * 0.3,
        viewport_height * 0.6,
        viewport_height * 0.9,
        scroll_height * 0.2,
        scroll_height * 0.4,
        scroll_height * 0.6,
        scroll_height * 0.8,
        scroll_height
    ]
    
    for i, position in enumerate(scroll_positions):
        # 随机滚动速度
        scroll_script = f"""
        window.scrollTo({{
            top: {position},
            behavior: 'smooth'
        }});
        """
        driver.execute_script(scroll_script)
        
        # 随机等待时间
        wait_time = random.uniform(0.5, 2.5)
        time.sleep(wait_time)
        
        # 偶尔随机小范围滚动
        if random.random() < 0.3:
            small_scroll = random.randint(-100, 100)
            driver.execute_script(f"window.scrollBy(0, {small_scroll})")
            time.sleep(0.5)

def deep_scroll_for_content(driver):
    """深度滚动以加载更多内容"""
    print("深度滚动加载内容...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    no_change_count = 0
    
    for i in range(10):  # 最多尝试10次
        # 滚动到底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # 检查是否有"Load More"按钮并点击
        if click_load_more_buttons(driver):
            time.sleep(3)
        
        # 检查高度是否变化
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            no_change_count += 1
            if no_change_count >= 3:
                print("连续3次滚动高度无变化，停止滚动")
                break
        else:
            no_change_count = 0
            print(f"发现新内容，页面高度: {new_height}")
        
        last_height = new_height
        
        # 随机等待
        time.sleep(random.uniform(1, 3))

def click_load_more_buttons(driver):
    """点击所有可能的加载更多按钮"""
    load_more_selectors = [
        "button[data-testid*='load-more']",
        "button:contains('Load More')",
        "button:contains('顯示更多')",
        "button:contains('Show More')",
        ".load-more",
        "[class*='load-more']"
    ]
    
    for selector in load_more_selectors:
        try:
            # 使用XPath包含文本的查找
            buttons = driver.find_elements(By.XPATH, 
                f"//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), '顯示更多')]")
            
            for button in buttons:
                try:
                    if button.is_displayed() and button.is_enabled():
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", button)
                        print("✅ 点击了加载更多按钮")
                        return True
                except:
                    continue
        except:
            continue
    
    return False

def extract_products_advanced(driver, seen_products):
    """高级商品提取"""
    products = []
    
    # 多种元素定位策略
    extraction_strategies = [
        extract_by_data_testid,
        extract_by_class_patterns,
        extract_by_link_analysis,
        extract_by_structural_analysis
    ]
    
    all_found_products = []
    
    for strategy in extraction_strategies:
        try:
            found_products = strategy(driver, seen_products)
            if found_products:
                all_found_products.extend(found_products)
                print(f"策略 {strategy.__name__} 找到 {len(found_products)} 个商品")
                
                # 如果找到很多商品，可以提前停止
                if len(all_found_products) > 50:
                    break
        except Exception as e:
            print(f"策略 {strategy.__name__} 失败: {e}")
            continue
    
    # 去重并添加到结果
    for product in all_found_products:
        product_id = create_product_id(product)
        if product_id not in seen_products:
            seen_products.add(product_id)
            products.append(product)
    
    return products

def extract_by_data_testid(driver, seen_products):
    """通过data-testid属性提取"""
    products = []
    
    testid_patterns = [
        'listing-card',
        'product-card',
        'listing',
        'product',
        'item'
    ]
    
    for pattern in testid_patterns:
        selector = f'[data-testid*="{pattern}"]'
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                product = extract_from_element(element)
                if product and product not in products:
                    products.append(product)
        except:
            continue
    
    return products

def extract_by_class_patterns(driver, seen_products):
    """通过类名模式提取"""
    products = []
    
    class_patterns = [
        'listingCard',
        'productCard',
        'listing',
        'card',
        'item'
    ]
    
    for pattern in class_patterns:
        selectors = [
            f'div[class*="{pattern}"]',
            f'article[class*="{pattern}"]',
            f'section[class*="{pattern}"]',
            f'a[class*="{pattern}"]'
        ]
        
        for selector in selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    product = extract_from_element(element)
                    if product and product not in products:
                        products.append(product)
            except:
                continue
    
    return products

def extract_by_link_analysis(driver, seen_products):
    """通过链接分析提取"""
    products = []
    
    try:
        # 查找所有商品链接
        link_elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
        
        for link_element in link_elements:
            try:
                # 获取链接周围的容器
                container = link_element
                for i in range(3):
                    try:
                        container = container.find_element(By.XPATH, "./..")
                        if container.text and len(container.text.strip()) > 30:
                            product = extract_from_element(container)
                            if product and product not in products:
                                products.append(product)
                            break
                    except:
                        break
            except:
                continue
                
    except Exception as e:
        print(f"链接分析失败: {e}")
    
    return products

def extract_by_structural_analysis(driver, seen_products):
    """通过结构分析提取"""
    products = []
    
    try:
        # 查找包含商品信息的结构块
        structural_selectors = [
            'div > div > div',  # 三层div结构
            'article > div',
            'section > div',
            '[role="listitem"]'
        ]
        
        for selector in structural_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    text = element.text.strip()
                    # 检查是否包含商品特征
                    if (len(text) > 50 and 
                        ('dimoo' in text.lower() or 'HK$' in text or '$' in text)):
                        product = extract_from_element(element)
                        if product and product not in products:
                            products.append(product)
            except:
                continue
                
    except Exception as e:
        print(f"结构分析失败: {e}")
    
    return products

def extract_from_element(element):
    """从元素提取商品信息"""
    try:
        text = element.text.strip()
        if len(text) < 30:
            return None
        
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        if len(lines) < 3:
            return None
        
        # 提取信息
        title = find_title(lines)
        price = find_price(lines)
        seller = find_seller(lines)
        time_text = find_time(lines)
        link = find_link(element)
        
        if not title:
            return None
            
        return {
            'title': clean_text(title),
            'price': price or '價格待詢',
            'seller': clean_text(seller) if seller else '',
            'post_time': clean_text(time_text) if time_text else '',
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
    except Exception as e:
        return None

def find_title(lines):
    """查找标题"""
    # 优先查找包含dimoo的行
    for line in lines:
        if 'dimoo' in line.lower() and 10 < len(line) < 100:
            return line
    
    # 返回最长的合适行
    candidate_lines = [line for line in lines if 20 < len(line) < 150]
    if candidate_lines:
        return max(candidate_lines, key=len)
    
    return lines[0] if lines else ""

def find_price(lines):
    """查找价格"""
    for line in lines:
        price_match = re.search(r'(HK\$|\$)\s*(\d+(?:,\d+)*(?:\.\d+)?)', line)
        if price_match:
            return f"{price_match.group(1)}{price_match.group(2)}"
    
    for line in lines:
        if 'free' in line.lower() or '免費' in line:
            return "FREE"
    
    return ""

def find_seller(lines):
    """查找卖家"""
    if len(lines) > 0:
        return lines[0]
    return ""

def find_time(lines):
    """查找时间"""
    time_patterns = [
        r'\d+\s*(分鐘|分钟|小時|小时|天|日|月|年)\s*前',
        r'\d+\s*(minute|hour|day|week|month|year)s?\s*ago',
        r'剛剛|剛才|just now'
    ]
    
    for line in lines:
        for pattern in time_patterns:
            if re.search(pattern, line, re.IGNORECASE):
                return line
    return ""

def find_link(element):
    """查找链接"""
    try:
        if element.tag_name == "a":
            return element.get_attribute("href")
        else:
            link_elem = element.find_element(By.CSS_SELECTOR, 'a[href*="/p/"]')
            return link_elem.get_attribute("href") if link_elem else ""
    except:
        return ""

def clean_text(text):
    """清理文本"""
    return re.sub(r'\s+', ' ', text).strip()

def has_more_content(driver):
    """检查是否还有更多内容"""
    try:
        # 检查加载更多按钮
        load_more_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), '顯示更多')]"
        )
        if load_more_buttons:
            return True
        
        # 检查下一页按钮
        next_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Next') or contains(text(), '下一頁')]"
        )
        if next_buttons:
            return True
            
        # 检查是否到达底部
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        client_height = driver.execute_script("return document.documentElement.clientHeight")
        scroll_top = driver.execute_script("return document.documentElement.scrollTop")
        
        if scroll_top + client_height < scroll_height - 100:
            return True
            
    except:
        pass
    
    return False

def smart_pagination(driver, current_page):
    """智能翻页"""
    try:
        # 方法1: 点击下一页按钮
        next_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Next') or contains(text(), '下一頁')]"
        )
        
        for button in next_buttons:
            try:
                if button.is_displayed() and button.is_enabled():
                    driver.execute_script("arguments[0].click();", button)
                    print("✅ 点击下一页按钮")
                    time.sleep(3)
                    return True
            except:
                continue
        
        # 方法2: URL翻页
        next_url = f"https://www.carousell.com.hk/search/dimoo?page={current_page + 1}"
        try:
            driver.get(next_url)
            print(f"✅ 跳转到第 {current_page + 1} 页")
            time.sleep(3)
            return True
        except:
            pass
            
        # 方法3: 滚动加载
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"翻页失败: {e}")
        return False

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info.get('link', '')}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def save_advanced_data(products):
    """保存高级数据"""
    if not products:
        return None
        
    try:
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = f"carousell_dimoo_advanced_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        filepath = os.path.join(desktop_path, filename)
        
        df = pd.DataFrame(products)
        
        # 列映射
        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户', 
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        
        df = df.rename(columns=column_mapping)
        available_columns = [col for col in column_mapping.values() if col in df.columns]
        df = df[available_columns]
        
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 高级数据已保存: {filepath}")
        print(f"📊 总数据量: {len(df)} 条记录")
        
        return filepath
        
    except Exception as e:
        print(f"保存失败: {e}")
        return None

def run_mass_scraper():
    """运行大规模爬取"""
    print("🚀 开始大规模爬取Carousell Molly商品...")
    print("🎯 目标: 突破500条限制，获取1000+条数据")
    
    all_products = []
    max_attempts = 3
    
    for attempt in range(max_attempts):
        print(f"\n{'='*60}")
        print(f"尝试 {attempt + 1}/{max_attempts}")
        print(f"{'='*60}")
        
        products, saved_file = scrape_carousell_advanced()
        
        if products:
            all_products.extend(products)
            print(f"✅ 本次获取 {len(products)} 条数据")
            print(f"📊 累计获取 {len(all_products)} 条数据")
            
            if len(all_products) >= 800:
                print("🎉 已达到目标数据量!")
                break
        else:
            print("❌ 本次爬取失败")
        
        # 长时间休息避免被封
        if attempt < max_attempts - 1:
            wait_time = random.randint(30, 60)
            print(f"⏰ 等待 {wait_time} 秒后继续...")
            time.sleep(wait_time)
    
    # 最终保存
    if all_products:
        final_file = save_advanced_data(all_products)
        print(f"\n🎉 最终结果: 成功获取 {len(all_products)} 条数据!")
        print(f"💾 保存到: {final_file}")
        return all_products, final_file
    else:
        print("❌ 所有尝试都失败了")
        return None, None

if __name__ == "__main__":
    # 运行大规模爬取
    products, saved_file = run_mass_scraper()

🚀 开始大规模爬取Carousell Molly商品...
🎯 目标: 突破500条限制，获取1000+条数据

尝试 1/3

🎯 尝试URL: https://www.carousell.com.hk/search/dimoo
访问: https://www.carousell.com.hk/search/dimoo
模拟人类滚动...
处理第 1 页...
深度滚动加载内容...
✅ 点击了加载更多按钮
发现新内容，页面高度: 12938
✅ 点击了加载更多按钮
发现新内容，页面高度: 19294
✅ 点击了加载更多按钮
发现新内容，页面高度: 25234
✅ 点击了加载更多按钮
发现新内容，页面高度: 31634
✅ 点击了加载更多按钮
发现新内容，页面高度: 37386
✅ 点击了加载更多按钮
发现新内容，页面高度: 43182
✅ 点击了加载更多按钮
发现新内容，页面高度: 49250
✅ 点击了加载更多按钮
发现新内容，页面高度: 55190
✅ 点击了加载更多按钮
发现新内容，页面高度: 60964
✅ 点击了加载更多按钮
发现新内容，页面高度: 67010
策略 extract_by_data_testid 找到 1054 个商品
第 1 页找到 527 个新商品
✅ 跳转到第 2 页
处理第 2 页...
深度滚动加载内容...
✅ 点击了加载更多按钮
发现新内容，页面高度: 12816
✅ 点击了加载更多按钮
发现新内容，页面高度: 18756
✅ 点击了加载更多按钮
发现新内容，页面高度: 25156
✅ 点击了加载更多按钮
发现新内容，页面高度: 30742
✅ 点击了加载更多按钮
发现新内容，页面高度: 36954
✅ 点击了加载更多按钮
发现新内容，页面高度: 42584
✅ 点击了加载更多按钮
发现新内容，页面高度: 48818
✅ 点击了加载更多按钮
发现新内容，页面高度: 54426
✅ 点击了加载更多按钮
发现新内容，页面高度: 60200
✅ 点击了加载更多按钮
发现新内容，页面高度: 65808
策略 extract_by_data_testid 找到 1050 个商品
第 2 页找到 525 个新商品
✅ 跳转到第 3 页
处理第 3 页...
深度滚动加载内容...
✅ 点击了加载更多按钮
发现新内容，页面高度: 128